# **(1.0)** Kinetic Modeling in Relation to Constraint-Based Approaches (FBA and MFA)

Thus far, we have dealt exclusively with kinetic models, looking at concentrations, fluxes, and labeling of chemical species under dynamic or steady-state conditions. A key limitation of kinetic modeling is the large number of parameters that it requires. If we think back to the reversible Michaelis-Menten with noncompetitive inhibition simulation we did in yesterday's session, one flux in the network was determined by the following equation:

$$ J_{ab} = \frac{Vmax_{AB}^{f}\frac{[A]}{Km_{AB}^{f}}-Vmax_{AB}^{b}\frac{[B]}{Km_{AB}^{b}}}{1+\frac{[A]}{Km_{AB}^{f}}+\frac{[B]}{Km_{AB}^{b}}+\frac{[I]}{Ki}} $$

That is a lot of parameters and without estimates of a significant number of them, we aren't going to get precise predictions of fluxes. Is there a way we can simplify our description of what's going on mathematically while still allowing us to extract flux estimates?

# **(1.1)** Steady State Assumption and Constraint-Based Analyses

The network we will look at for today's exercise is shown below.

![alt text](NetworkDiagram_1.png "Title")

Note that this features a cycle involving the metabolites B, C, and D. Also note that metabolites A and B are non-symmetrical molecules consisting of two atoms that can be labeled, whereas D and C contain one atom. If we introduce a labeled atom in the position shown in the above figure via metabolite A, this labeled atom will be in the same position in metabolite B and then move into metabolite D via reaction R3. Label can move from D to C via the reverse reaction R4. We can represent this network architecture using a stoichiometric matrix, shown below:

![alt text](StoichMatrix.jpg "Title")

We can relate our stoichiometric matrix above to our fluxes and concentrations as shown below:

![alt text](StoichiometryEquation.png "Title")

Where **S** is our stoichiometric matrix, **v** is our vector of fluxes, and **c** is a vector of differential equations describing the change in the concentrations of the metabolites of our network over time. When we are doing kinetic modeling, we provide sets of parameters and initial concentrations that allow us to calculate fluxes which, in combination with the network architecture given by the stoichiometric matrix, give us the rates of change of the concentrations of our metabolites. This gives us a rich and detailed view of the dynamics of the system we're looking at and, importantly, it is a very attractive way of modeling biochemical systems because it hews closely to our understanding of how these systems *actually* work. But, as we just highlighted, this is extremely demanding in terms of the number of parameters we need to know the values of. 

Recall that in the metabolic steady state, our metabolite concentrations are not changing. That is ...



![alt text](StoichiometryEquation_SteadyState.png "Title")

Making this simplification allows for **constraint-based** analyses, of which the two types we'll cover are **Flux Balance Analysis** (FBA) and **Metabolic Flux Analysis** (MFA). In the following exercises, we'll get a feel for the basic applications and limitations of both of these techniques. 

# **(1.2)** Kinetic Approach

To start off, let's simulate fluxes through this network using reversible and irreversible first-order kinetics and a defined set of parameters. Metabolites A and B have two positions that can be labeled. We denote fully labeled molecules with "11", fully unlabeled molecules with "00", and the two half-labeled configurations with "10" and "01". Sliders to set the initial concentrations of all these species can be found in the interface below.

<div class="alert alert-block alert-info">

**Instructions:** Run the three cells below. When you run the interactive cell, you will get a print-out of the steady-state fluxes of every reaction in our network along with the fractional labeling of each metabolite at steady-state. The conditions we're going to look at are 
    
1. The default parameters the interactive cell starts with, except with the R1 10 fraction parameter set to 0.
2. The default parameters the interactive cell starts with.
3. The default parameters, except with the R4 rate constant set to 0.5 and the R5 rate constant set to 0.5.
    
Record the flux and fractional labeling values you get from each of these three scenarios somewhere. We will be comparing these "ground truth" results to our FBA and MFA results.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are importing the Python packages we'll need for today's exercises.
#-----------------------------------------------------------------------------------------------------------------------

from pprint import pprint           # Imports a function that can print out our results in an easier-to-read format

import numpy as np                  # Imports numpy for handling arrays of numbers

import math                         # Imports the math package to handle a few stand ard math operations we'll be doing                                                

import seaborn as sns               # Imports the seaborn package, which allows us to make some more 
                                    # aesthetically pleasing plots

import matplotlib.pyplot as plt     # Imports the matplotlib package (specifically pyplot), which we'll be using 
                                    # to generate our plots

import ipywidgets as widgets        # Imports the ipywidgets package, which we'll use for interactivity

from IPython.display import display, Markdown, clear_output             # We import some additional elements we'll need
                                                                        # for our interactive code

from ipywidgets.widgets.interaction import show_inline_matplotlib_plots # We'll use this to make sure our plots
                                                                        # that we get from our interactive widgets
                                                                        # display correctly

import pandas as pd                 # Imports the pandas package, which we will use for importing and 
                                    # exporting dataframes 

import mfapy                        # Imports the mfapy package for 13C-MFA analysis

import cobra as cb                                        # Imports the cobrapy package to handle FBA modeling
from cobra.sampling import sample                         # Imports the sampling function from cobra
from cobra.flux_analysis import flux_variability_analysis # Imports the flux variability analysis function from cobra

import scipy.stats as stats

import os, sys, time, copy, csv

import numpy

%matplotlib inline                  

sns.set()                           # This makes our plots look more aesthetically pleasing

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation, as well as the function that carries
# out the simulation itself. Note that we track the positional labeling of metabolites A and B by independently tracking
# the four isotopomers possible with a two carbon compound. We denote these as m00, m10, m01, and m11, with zeroes representing
# unlabeled positions and ones representing labeled positions.
#-----------------------------------------------------------------------------------------------------------------------

# Function for flux evaluation

def evaluate_flux(k, S):                                            
    flux = k * S                                                   
    return flux                                                    

def evaluate_flux_bimolecular(k, S, S2):                           # Defining a version of our evaluate_flux() function that
                                                                   # calculates fluxes resulting from bimolecular reactions - 
                                                                   # that is, reactions whose rate will be proportional to the 
                                                                   # rate constant times the concentrations of two different species
    '''                                                            
    DESCRIPTION
    
    Takes a rate constant k and a substrate concentration S 
    and returns a flux value.
    
    INPUTS
    
    k = a numeric first order rate constant
    
    S = a numeric concentration of a substrate
    
    S2 = a numeric concentration of a substrate
    
    OUTPUTS
    
    flux = flux, or reaction rate, given substrate 
    concentration and a first order rate constant
    '''
    
    flux = k * S * S2                                              # Defining flux as the product of the provided
                                                                   # rate constant and the two relevant substrate concentrations
            
    return flux                                                    

#-----------------------------------------------------------------------------------------------------------------------
# Now we define functions to evaluate the rate of change of the metabolites in our simulation. 
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_dAdt_f(R1, R2, f_m00_R1, f_m10_R1, f_m01_R1, f_m11_R1, f_m00_A, f_m10_A, f_m01_A, f_m11_A): # Done
    '''
    DESCRIPTION
    
    Takes the fluxes into out and out of metabolite A (Jsa and Jab)
    in the forward directionand the fractional labeling of S and A 
    to calculate the change in concentration of either labeled or unlabeled A.
    
    INPUTS
    
    R1 = a numeric flux value R1
    
    R2 = a numeric flux value R2
    
    f_m00_A = the fraction of metabolite R1 with 0 labeled and two unlabeled positions
    
    f_m10_A = the fraction of metabolite R1 with 1 labeled and 1 unlabeled positions
    
    f_m01_A = the fraction of metabolite R1 with 2 labeled and 0 unlabeled positions
    
    f_m11_A = the fraction of metabolite R1 with 2 labeled and 0 unlabeled positions
    
    f_m00_A = the unlabeled fraction labeling of incoming flux R1 
    
    f_m10_A = the fraction of incoming flux R1 that is labeled at the first position and unlabeled at the second position
    
    f_m10_A = the fraction of incoming flux R1 that is unlabeled at the first position and labeled at the second position
    
    f_m11_A = the fully labeled fraction of incoming flux R1

    
    OUTPUTS
    
    dLAdt = the change in the labeled or unlabeled concentration
    of A
    
    '''
    
    dm00dt = (R1 * f_m00_R1) - (R2 * f_m00_A)              # We calculate the change in the concentration of each species 
    dm10dt = (R1 * f_m10_R1) - (R2 * f_m10_A)              # independently
    dm01dt = (R1 * f_m01_R1) - (R2 * f_m01_A)
    dm11dt = (R1 * f_m11_R1) - (R2 * f_m11_A)
    
    return dm00dt, dm10dt, dm01dt, dm11dt                  # and return the changes in all four concentrations

def evaluate_dBdt_f(R2, R3, f_m00_A, f_m10_A, f_m01_A, f_m11_A, f_m00_B, f_m10_B, f_m01_B, f_m11_B):
    
    dm00dt = (R2 * f_m00_A) - (R3 * f_m00_B)
    dm10dt = (R2 * f_m10_A) - (R3 * f_m10_B)
    dm01dt = (R2 * f_m01_A) - (R3 * f_m01_B)
    dm11dt = (R2 * f_m11_A) - (R3 * f_m11_B)

    return dm00dt, dm10dt, dm01dt, dm11dt

def evaluate_dBdt_b(R3_b, f_m0_D, f_m1_D, f_m0_C, f_m1_C): 
    
    dm00dt = (R3_b * f_m0_C * f_m0_D)
    dm10dt = (R3_b * f_m0_C * f_m1_D)
    dm01dt = (R3_b * f_m1_C * f_m0_D)   
    dm11dt = (R3_b * f_m1_C * f_m1_D)   
    
    return dm00dt, dm10dt, dm01dt, dm11dt

def evaluate_dCdt_f(R3, R4, R5, f_m00_B, f_m10_B, f_m01_B, f_m11_B, f_m0_C, f_m1_C): 
    
    dm0dt = (R3 * f_m00_B) + (R3 * f_m10_B) - (R4 * f_m0_C) - (R5 * f_m0_C)
    dm1dt = (R3 * f_m01_B) + (R3 * f_m11_B) - (R4 * f_m1_C) - (R5 * f_m1_C)
    
    return dm0dt, dm1dt

def evaluate_dCdt_b(R3_b, R4_b, f_m0_C, f_m1_C, f_m0_D, f_m1_D): 
    
    dm0dt = - (R3_b * f_m0_C) + (R4_b * f_m0_D)
    dm1dt = - (R3_b * f_m1_C) + (R4_b * f_m1_D)
    
    return dm0dt, dm1dt

def evaluate_dDdt_f(R3, R4, f_m00_B, f_m10_B, f_m01_B, f_m11_B, f_m0_C, f_m1_C): 
    
    dm0dt = (R3 * f_m00_B) + (R3 * f_m01_B) + (R4 * f_m0_C)
    dm1dt = (R3 * f_m10_B) + (R3 * f_m11_B) + (R4 * f_m1_C) 
    
    return dm0dt, dm1dt

def evaluate_dDdt_b(R3_b, R4_b, f_m0_D, f_m1_D):

    dm0dt = - (R3_b * f_m0_D) - (R4_b * f_m0_D)
    dm1dt = - (R3_b * f_m1_D) - (R4_b * f_m1_D)
    
    return dm0dt, dm1dt

def evaluate_fraction_B(F_00, F_10, F_01, F_11):                 # Calculating and returning the 00, 10, 01 and 11 fractions
    F_total = F_00 + F_10 + F_01 + F_11                          # of a two-carbon metabolite
    if F_total == 0:
        fraction_00 = 0
        fraction_10 = 0
        fraction_01 = 0
        fraction_11 = 0
    else:
        fraction_00 = F_00 / F_total
        fraction_10 = F_10 / F_total
        fraction_01 = F_01 / F_total
        fraction_11 = F_11 / F_total
    return fraction_00, fraction_10, fraction_01, fraction_11

def evaluate_fraction(Fm0, Fm1):                        # Calculating and returning the m0 and m1 fraction
    F_total = Fm0 + Fm1                                 # of a one-carbon metabolite
    if F_total == 0:
        fraction_m0 = 0
        fraction_m1 = 0
    else:
        fraction_m0 = Fm0 / F_total
        fraction_m1 = Fm1 / F_total
    return fraction_m0, fraction_m1

#-----------------------------------------------------------------------------------------------------------------------
# Now that we have all of these functions defined, we can define the function that will actually run our simulation
# and give us the results we're going to analyze. 
#-----------------------------------------------------------------------------------------------------------------------

def kinetic_simulation(total_time = 1000, timestep = 1, m00_A = 0, m10_A = 0, m01_A = 0, m11_A = 0, m00_B = 0, m10_B = 0, m01_B = 0, m11_B = 0, m0_C = 0, m1_C = 0, m0_D = 0, m1_D = 0,  
        R1 = 10, K_R2 = 0.1, K_R3 = 0.1, K_R3_b = 0.01, K_R4 = 0.1, K_R4_b = 0.01, K_R5 = 0.1, f_m00_R1 = 0, f_m10_R1 = 1, f_m01_R1 = 0, f_m11_R1 = 0):
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using reversible first-order kinetics.
    Accepts arguments to both export generated data and import another group's data to 
    compare with simulation results. Can also subset generated data and add random noise
    before exporting.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    
    m00_A = A numeric value representing the initial concentration of m00 A
    
    m10_A = A numeric value representing the initial concentration of m10 A
    
    m01_A = A numeric value representing the initial concentration of m01 A
    
    m11_A = A numeric value representing the initial concentration of m11 A
    
    m00_B = A numeric value representing the initial concentration of m00 B
    
    m10_B = A numeric value representing the initial concentration of m10 B
    
    m01_B = A numeric value representing the initial concentration of m01 B

    m11_B = A numeric value representing the initial concentration of m11 B
    
    m0_C = A numeric value representing the initial concentration of m0 C
    
    m1_C = A numeric value representing the initial concentration of m1 C
    
    m0_D = A numeric value representing the initial concentration of m0 D
    
    m1_D = A numeric value representing the initial concentration of m1 D   
    
    R1 = A numeric value representing the flux through reaction R1
    
    K_R2 = A numeric value representing the first-order rate constant for the
    reaction R2
    
    K_R3 = A numeric value representing the first-order rate constant for the
    reaction R3
    
    K_R3_b = A numeric value representing the first-order rate constant for the
    reaction R3_b
    
    K_R4 = A numeric value representing the first-order rate constant for the
    reaction R4
    
    K_R4_b = A numeric value representing the first-order rate constant for the
    reaction R4_b
    
    K_R5 = A numeric value representing the first-order rate constant for the
    reaction R5

    f_m00_R1 = A numeric value representing the fraction of R1 that is m00
    
    f_m10_R1 = A numeric value representing the fraction of R1 that is m10
    
    f_m01_R1 = A numeric value representing the fraction of R1 that is m01
    
    f_m11_R1 = A numeric value representing the fraction of R1 that is m11
    
    OUTPUTS
    
    None
    
    '''
    
    #----------------------------------------------------------------------------------------------------------------
    # Initial Simulation setup
    #----------------------------------------------------------------------------------------------------------------

    # Creating the timepoints we'll be using
    
    times = np.arange(0, total_time, timestep)                    # Here, we use the total_time parameter together with our
                                                                  # timestep parameter to generate a list of timepoints.

    # Calculating our initial total concentrations
    
    A_total = m00_A + m10_A + m01_A + m11_A                      # We need to know our starting TOTAL concentrations, not just
    B_total = m00_B + m10_B + m01_B + m11_B                      # the unlabeled and labeled concentrations, so we calculate these
    C_total = m0_C + m1_C
    D_total = m0_C + m1_C

     # Creating our lists #
    
    # Lists for the unlabeled substrate concentrations
    
    m00_A_list = [m00_A]                                                  # We need to keep track of the values we care about at each time                                                    
    m00_B_list = [m00_B]                                                  # time point, so here we start making lists. During our simulation                                                 
    m0_C_list = [m0_C]                                                    # we'll add a new value onto each of these lists every time step.
    m0_D_list = [m0_D]                                                    # Here, we're making the ones for unlabeled concentrations and 
                                                                          # function accepts.
    m10_A_list = [m10_A]                                                  # We need to keep track of the values we care about at each time                                                    
    m10_B_list = [m10_B]                                                  # time point, so here we start making lists. During our simulation                                                 
    m1_C_list = [m1_C]                                                    # we'll add a new value onto each of these lists every time step.
    m1_D_list = [m1_D]  
    
    m01_A_list = [m01_A]                                                    # We need to keep track of the values we care about at each time                                                    
    m01_B_list = [m01_B]                                                    # time point, so here we start making lists. During our simulation                                                 
                                                                            # we'll add a new value onto each of these lists every time step.
    m11_A_list = [m11_A]
    m11_B_list = [m11_B]
    
    
    # Lists for total concentrations
    
    A_total_list = [A_total]                                     # Same thing, but for the total concentrations
    B_total_list = [B_total]
    C_total_list = [C_total]
    D_total_list = [D_total]
    
    # Lists for the labeling of substrate classes
    
    f_m00_A, f_m10_A, f_m01_A, f_m11_A = evaluate_fraction_B(m00_A, m10_A, m01_A, m11_A)
    f_m00_B, f_m10_B, f_m01_B, f_m11_B = evaluate_fraction_B(m00_B, m10_B, m01_B, m11_B)
    f_m0_C, f_m1_C = evaluate_fraction(m0_C, m1_C)
    f_m0_D, f_m1_D = evaluate_fraction(m0_D, m1_D)
    
    f_m00_A_list = [f_m00_A]
    f_m10_A_list = [f_m10_A]
    f_m01_A_list = [f_m01_A]
    f_m11_A_list = [f_m11_A]
    f_m00_B_list = [f_m00_B]
    f_m10_B_list = [f_m10_B]
    f_m01_B_list = [f_m01_B]
    f_m11_B_list = [f_m11_B]
    f_m0_C_list = [f_m0_C]
    f_m1_C_list = [f_m1_C]
    f_m0_D_list = [f_m0_D]
    f_m1_D_list = [f_m1_D]  
    f_m00_R1_list = [f_m00_R1]
    f_m10_R1_list = [f_m10_R1]
    f_m01_R1_list = [f_m01_R1]
    f_m11_R1_list = [f_m11_R1]
    
    f_m1_A_list = [f_m10_A + f_m01_A]
    f_m1_B_list = [f_m10_B + f_m01_B]
    

    # Lists for flux values
    
    R1_list = [R1]            # Same thing, but for fluxes
    R2_list = [evaluate_flux(K_R2, A_total_list[-1])]
    R3_list = [evaluate_flux(K_R3, B_total_list[-1])]
    R3_b_list = [evaluate_flux_bimolecular(K_R3_b, C_total_list[-1], D_total_list[-1])]
    R4_list = [evaluate_flux(K_R4, C_total_list[-1])]
    R4_b_list = [evaluate_flux(K_R4_b, D_total_list[-1])]
    R5_list = [evaluate_flux(K_R5, C_total_list[-1])]


    #----------------------------------------------------------------------------------------------------------------
    # Starting the simulation
    #----------------------------------------------------------------------------------------------------------------    
    
    for i in times:                                              

        #------------------------------------------------------------------------------------------------------------
        # Evaluating current fluxes, changes in labeled/unlabeled/total concentrations, and fractional labeling
        #------------------------------------------------------------------------------------------------------------
        
        current_R1 = R1            
        current_R2 = evaluate_flux(K_R2, A_total_list[-1])
        current_R3 = evaluate_flux(K_R3, B_total_list[-1])
        current_R3_b = evaluate_flux_bimolecular(K_R3_b, C_total_list[-1], D_total_list[-1])  # This flux requires the evaluate_flux_bimolecular()
        current_R4 = evaluate_flux(K_R4, C_total_list[-1])                                    # function that we defined earlier
        current_R4_b = evaluate_flux(K_R4_b, D_total_list[-1])
        current_R5 = evaluate_flux(K_R5,C_total_list[-1])

        # Below, we evaluate the changes to the concentration of each individual m00, m10, m01, and m11 species in the
        # simulation based on the forward reactions and update the relevant lists with the new concentrations
        
        dm00_A, dm10_A, dm01_A, dm11_A = evaluate_dAdt_f(current_R1, current_R2, f_m00_R1_list[-1], f_m10_R1_list[-1], f_m01_R1_list[-1], f_m11_R1_list[-1], f_m00_A_list[-1], f_m10_A_list[-1], f_m01_A_list[-1], f_m11_A_list[-1])
        dm00_B, dm10_B, dm01_B, dm11_B = evaluate_dBdt_f(current_R2, current_R3, f_m00_A_list[-1], f_m10_A_list[-1], f_m01_A_list[-1], f_m11_A_list[-1], f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1])
        dm0_C, dm1_C = evaluate_dCdt_f(current_R3, current_R4, current_R5, f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1], f_m0_C_list[-1], f_m1_C_list[-1])
        dm0_D, dm1_D = evaluate_dDdt_f(current_R3, current_R4, f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1], f_m0_C_list[-1], f_m1_C_list[-1])                                     
        
        current_m00_A = m00_A_list[-1] + dm00_A * timestep
        current_m10_A = m10_A_list[-1] + dm10_A * timestep
        current_m01_A = m01_A_list[-1] + dm01_A * timestep
        current_m11_A = m11_A_list[-1] + dm11_A * timestep

        current_m00_B = m00_B_list[-1] + dm00_B * timestep
        current_m10_B = m10_B_list[-1] + dm10_B * timestep
        current_m01_B = m01_B_list[-1] + dm01_B * timestep
        current_m11_B = m11_B_list[-1] + dm11_B * timestep

        current_m0_C = m0_C_list[-1] + dm0_C * timestep
        current_m1_C = m1_C_list[-1] + dm1_C * timestep
        current_m0_D = m0_D_list[-1] + dm0_D * timestep
        current_m1_D = m1_D_list[-1] + dm1_D * timestep
        
        current_A_total = current_m00_A + current_m10_A + current_m01_A + current_m11_A
        current_B_total = current_m00_B + current_m10_B + current_m01_B + current_m11_A
        current_C_total = current_m0_C + current_m1_C
        current_D_total = current_m0_D + current_m1_D
        
        current_f_m00_A, current_f_m10_A, current_f_m01_A, current_f_m11_A = evaluate_fraction_B(current_m00_A, current_m10_A, current_m01_A, current_m11_A)
        current_f_m00_B, current_f_m10_B, current_f_m01_B, current_f_m11_B = evaluate_fraction_B(current_m00_B, current_m10_B, current_m01_B, current_m11_B)
        current_f_m0_C, current_f_m1_C = evaluate_fraction(current_m0_C, current_m1_C)
        current_f_m0_D, current_f_m1_D = evaluate_fraction(current_m0_D, current_m1_D)
        current_f_m00_R1 = f_m00_R1
        current_f_m10_R1 = f_m10_R1
        current_f_m01_R1 = f_m01_R1
        current_f_m11_R1 = f_m11_R1

        # We repeat the above steps, but in the reverse direction
                
        dm00_B, dm10_B, dm01_B, dm11_B = evaluate_dBdt_b(current_R3_b, current_f_m0_D, current_f_m1_D, current_f_m0_C, current_f_m1_C)
        dm0_C, dm1_C = evaluate_dCdt_b(current_R3_b, current_R4_b, current_f_m0_C, current_f_m1_C, current_f_m0_D, current_f_m1_D)
        dm0_D, dm1_D = evaluate_dDdt_b(current_R3_b, current_R4_b, current_f_m0_D, current_f_m1_D)

        current_m00_A = current_m00_A
        current_m10_A = current_m10_A
        current_m01_A = current_m01_A
        current_m11_A = current_m11_A
        current_m00_B = current_m00_B + dm00_B * timestep
        current_m10_B = current_m10_B + dm10_B * timestep
        current_m01_B = current_m01_B + dm01_B * timestep
        current_m11_B = current_m11_B + dm11_B * timestep
        current_m0_C = current_m0_C + dm0_C * timestep
        current_m1_C = current_m1_C + dm1_C * timestep
        current_m0_D = current_m0_D + dm0_D * timestep
        current_m1_D = current_m1_D + dm1_D * timestep
                                       
        current_A_total = current_m00_A + current_m10_A + current_m01_A + current_m11_A
        current_B_total = current_m00_B + current_m10_B + current_m01_B + current_m11_A
        current_C_total = current_m0_C + current_m1_C
        current_D_total = current_m0_D + current_m1_D          
                
        current_f_m00_A, current_f_m10_A, current_f_m01_A, current_f_m11_A = evaluate_fraction_B(current_m00_A, current_m10_A, current_m01_A, current_m11_A)
        current_f_m00_B, current_f_m10_B, current_f_m01_B, current_f_m11_B = evaluate_fraction_B(current_m00_B, current_m10_B, current_m01_B, current_m11_B)
        current_f_m0_C, current_f_m1_C = evaluate_fraction(current_m0_C, current_m1_C)
        current_f_m0_D, current_f_m1_D = evaluate_fraction(current_m0_D, current_m1_D)
        current_f_m00_R1 = f_m00_R1
        current_f_m10_R1 = f_m10_R1
        current_f_m01_R1 = f_m01_R1
        current_f_m11_R1 = f_m11_R1

        #------------------------------------------------------------------------------------------------------------
        # Now that we've calculated our current concentrations, fluxes, and fractional labeling values, we can update
        # our lists with these most recent values. If the simulation hasn't ended, these will be used as the basis for
        # the next timestep's calculations.
        #------------------------------------------------------------------------------------------------------------
        
        m00_A_list.append(current_m00_A)
        m10_A_list.append(current_m10_A)
        m01_A_list.append(current_m01_A)
        m11_A_list.append(current_m11_A)
        m00_B_list.append(current_m00_B)
        m10_B_list.append(current_m10_B)
        m01_B_list.append(current_m01_B)
        m11_B_list.append(current_m11_B)
        m0_C_list.append(current_m0_C)
        m1_C_list.append(current_m1_C)
        m0_D_list.append(current_m0_D)
        m1_D_list.append(current_m1_D)
                                
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        C_total_list.append(current_C_total)
        D_total_list.append(current_D_total)
          
        f_m00_A_list.append(current_f_m00_A)
        f_m10_A_list.append(current_f_m10_A)
        f_m01_A_list.append(current_f_m01_A)
        f_m11_A_list.append(current_f_m11_A)

        f_m00_B_list.append(current_f_m00_B)
        f_m10_B_list.append(current_f_m10_B)
        f_m01_B_list.append(current_f_m01_B)
        f_m11_B_list.append(current_f_m11_B)

        f_m0_C_list.append(current_f_m0_C)
        f_m1_C_list.append(current_f_m1_C)
        f_m0_D_list.append(current_f_m0_D)
        f_m1_D_list.append(current_f_m1_D)  
        f_m00_R1_list.append(f_m00_R1)
        f_m10_R1_list.append(f_m10_R1)
        f_m01_R1_list.append(f_m01_R1)
        f_m11_R1_list.append(f_m11_R1)
        
        f_m1_A_list.append(current_f_m10_A + current_f_m01_A)
        f_m1_B_list.append(current_f_m10_B + current_f_m01_B)
                                
        R1_list.append(current_R1)
        R2_list.append(current_R2)
        R3_list.append(current_R3)
        R3_b_list.append(current_R3_b)
        R4_list.append(current_R4)
        R4_b_list.append(current_R4_b)
        R5_list.append(current_R5)
    
    R1_net = np.array(R1_list)
    R2_net = np.array(R2_list)
    R3_net = np.array(R3_list) - np.array(R3_b_list)                   # We calculated forward and reverse fluxes, but 
    R4_net = np.array(R4_list) - np.array(R4_b_list)                   # we may also be interested in seeing the net fluxes,
    R5_net = np.array(R5_list)                                         # so we retrieve those numbers by converting to numpy arrays

                                
    times_including_initial = np.insert(times, 0, times[0] - timestep)  
    
    #----------------------------------------------------------------------------------------------------------------
    # Now we plot our results
    #----------------------------------------------------------------------------------------------------------------    
        
    # Total concentration plot

    fig, ax = plt.subplots(1, 4, figsize = (16, 5))                    

    ax[0].plot(times_including_initial, A_total_list, label = 'A')       
    ax[0].plot(times_including_initial, B_total_list, label = 'B')    
    ax[0].plot(times_including_initial, C_total_list, label = 'C')     
    ax[0].plot(times_including_initial, D_total_list, label = 'D')       

    ax[0].legend()                                                       
    ax[0].set_title('Total Concentration Over Time', fontweight = 'bold')
    ax[0].set_xlabel('Time', fontweight = 'bold')                        
    ax[0].set_ylabel('Total Concentration', fontweight = 'bold')         

    # Plotting net fluxes

    ax[1].plot(times_including_initial, R1_net, label = 'R1')           
    ax[1].plot(times_including_initial, R2_net, label = 'R2')        
    ax[1].plot(times_including_initial, R3_net, label = 'R3')       
    ax[1].plot(times_including_initial, R4_net, label = 'R4')          
    ax[1].plot(times_including_initial, R5_net, label = 'R5')          
    
    ax[1].legend()                                                     
    ax[1].set_title('Fluxes', fontweight = 'bold')                      
    ax[1].set_xlabel('Time', fontweight = 'bold')                       
    ax[1].set_ylabel('Flux', fontweight = 'bold')                       

    #Plotting all fluxes

    ax[2].plot(times_including_initial, R1_list, label = 'R1')        
    ax[2].plot(times_including_initial, R2_list, label = 'R2')        
    ax[2].plot(times_including_initial, R3_list, label = 'R3')         
    ax[2].plot(times_including_initial, R3_b_list, label = 'R3_b')      
    ax[2].plot(times_including_initial, R4_list, label = 'R4')        
    ax[2].plot(times_including_initial, R4_b_list, label = 'R4_b')     
    ax[2].plot(times_including_initial, R5_list, label = 'R5')        

    ax[2].legend()                                                      
    ax[2].set_title('Fluxes', fontweight = 'bold')                       
    ax[2].set_xlabel('Time', fontweight = 'bold')                       
    ax[2].set_ylabel('Flux', fontweight = 'bold')                       

    # Plotting fractional labeling

    ax[3].plot(times_including_initial, f_m00_A_list, label = 'A m0')     
    ax[3].plot(times_including_initial, f_m1_A_list, label = 'A m1')     
    ax[3].plot(times_including_initial, f_m11_A_list, label = 'A m2')     
    ax[3].plot(times_including_initial, f_m00_B_list, label = 'B m0')     
    ax[3].plot(times_including_initial, f_m1_B_list, label = 'B m1')    
    ax[3].plot(times_including_initial, f_m11_B_list, label = 'B m2')    
    ax[3].plot(times_including_initial, f_m0_C_list, label = 'C m0')     
    ax[3].plot(times_including_initial, f_m1_C_list, label = 'C m1')     
    ax[3].plot(times_including_initial, f_m0_D_list, label = 'D m0')     
    ax[3].plot(times_including_initial, f_m1_D_list, label = 'D m1')     

    ax[3].legend()                                                      
    ax[3].set_title('Fraction Labeled Over Time', fontweight = 'bold')  
    ax[3].set_xlabel('Time', fontweight = 'bold')                       
    ax[3].set_ylabel('Total Concentration', fontweight = 'bold')         
    
    results_dict = {'R1 flux':R1_list[-1],
                    'R2 flux':R2_list[-1],
                    'R3 forward flux':R3_list[-1],
                    'R3 reverse flux':R3_b_list[-1],
                    'R3 net flux':R3_net[-1],
                    'R4 forward flux':R4_list[-1],
                    'R4 reverse flux':R4_b_list[-1],
                    'R4 net flux':R4_net[-1],                    
                    'R5 forward flux':R5_list[-1],
                    'R5 net flux':R5_net[-1],
                    'A m0':f_m00_A_list[-1],
                    'A m1':f_m1_A_list[-1],
                    'A m2':f_m11_A_list[-1],
                    'B m0':f_m00_B_list[-1],
                    'B m1':f_m1_B_list[-1],
                    'B m2':f_m11_B_list[-1],
                    'C m0':f_m0_C_list[-1],
                    'C m1':f_m1_C_list[-1],
                    'D m0':f_m0_D_list[-1],
                    'D m1':f_m1_D_list[-1]}

    pprint(results_dict)


In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements(total_time_slider_value = 1000, timestep_slider_value = 0.1,                   
                       m00_A_slider_value = 0, m10_A_slider_value = 0, m01_A_slider_value = 0, m11_A_slider_value = 0,
                       m00_B_slider_value = 0, m10_B_slider_value = 0, m01_B_slider_value = 0, m11_B_slider_value = 0,
                       m0_C_slider_value = 0, m1_C_slider_value = 0,
                       m0_D_slider_value = 0, m1_D_slider_value = 0,
                       R1_slider_value = 10,
                       K_R2_slider_value = 0.1, K_R3_slider_value = 0.1, K_R3_b_slider_value = 0.001,
                       K_R4_slider_value = 0.01, K_R4_b_slider_value = 0.1,
                       K_R5_slider_value = 0.1, f_m00_R1_slider_value = 0, f_m10_R1_slider_value = 1, f_m01_R1_slider_value = 0, f_m11_R1_slider_value = 0):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 10000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'm00_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m00_A_slider_value),
                              'm10_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m10_A_slider_value),
                              'm01_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m01_A_slider_value),
                              'm11_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m11_A_slider_value),
                              'm00_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m00_B_slider_value),
                              'm10_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m10_B_slider_value),
                              'm01_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m01_B_slider_value),
                              'm11_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m11_B_slider_value),
                              'm0_C_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m0_C_slider_value),
                              'm1_C_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m1_C_slider_value),
                              'm0_D_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m0_D_slider_value),
                              'm1_D_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m1_D_slider_value),
                              'R1_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = R1_slider_value),
                              'K_R2_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R2_slider_value),                              
                              'K_R3_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R3_slider_value),
                              'K_R3_b_slider':widgets.FloatSlider(min = 0, max = 0.01, step = 0.001, value = K_R3_b_slider_value),
                              'K_R4_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R4_slider_value),
                              'K_R4_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R4_b_slider_value),
                              'K_R5_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R5_slider_value),
                              'f_m00_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m00_R1_slider_value),
                              'f_m10_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m10_R1_slider_value),
                              'f_m01_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m01_R1_slider_value),
                              'f_m11_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m11_R1_slider_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements


def on_button_clicked(_):                          
    with out:
        clear_output()
        kinetic_simulation_interactive()
        show_inline_matplotlib_plots()
        
def kinetic_simulation_interactive():                             
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    m00_A = elements['m00_A_slider'].value
    m10_A = elements['m10_A_slider'].value
    m01_A = elements['m01_A_slider'].value
    m11_A = elements['m11_A_slider'].value
    m00_B = elements['m00_B_slider'].value
    m10_B = elements['m10_B_slider'].value
    m01_B = elements['m01_B_slider'].value
    m11_B = elements['m11_B_slider'].value
    m0_C = elements['m0_C_slider'].value
    m1_C = elements['m1_C_slider'].value
    m0_D = elements['m0_D_slider'].value
    m1_D = elements['m1_D_slider'].value
    R1 = elements['R1_slider'].value
    K_R2 = elements['K_R2_slider'].value
    K_R3 = elements['K_R3_slider'].value
    K_R3_b = elements['K_R3_b_slider'].value
    K_R4 = elements['K_R4_slider'].value
    K_R4_b = elements['K_R4_b_slider'].value
    K_R5 = elements['K_R5_slider'].value
    f_m00_R1 = elements['f_m00_R1_slider'].value
    f_m10_R1 = elements['f_m10_R1_slider'].value
    f_m01_R1 = elements['f_m01_R1_slider'].value
    f_m11_R1 = elements['f_m11_R1_slider'].value


    kinetic_simulation(total_time, timestep, m00_A, m10_A, m01_A, m11_A, m00_B, m10_B, m01_B, m11_B, m0_C, m1_C, m0_D, m1_D,  
        R1, K_R2, K_R3, K_R3_b, K_R4, K_R4_b,
        K_R5, f_m00_R1, f_m10_R1, f_m01_R1, f_m11_R1)    

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements()                      

elements['button'].on_click(on_button_clicked)        
                                                       
out = widgets.Output()                                

grid = widgets.GridspecLayout(9, 6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Initial [A 00]</b>')
grid[0,5] = elements['m00_A_slider']

grid[1,0] = widgets.HTML('<b>Initial [A 10]</b>')
grid[1,1] = elements['m10_A_slider']
grid[1,2] = widgets.HTML('<b>Initial [A 01]</b>')
grid[1,3] = elements['m01_A_slider']
grid[1,4] = widgets.HTML('<b>Initial [A 11]</b>')
grid[1,5] = elements['m11_A_slider']

grid[2,0] = widgets.HTML('<b>Initial [B 00]</b>')
grid[2,1] = elements['m00_B_slider']
grid[2,2] = widgets.HTML('<b>Initial [B 10]</b>')
grid[2,3] = elements['m10_B_slider']
grid[2,4] = widgets.HTML('<b>Initial [B 01]</b>')
grid[2,5] = elements['m01_B_slider']

grid[3,0] = widgets.HTML('<b>Initial [B 11]</b>')
grid[3,1] = elements['m11_B_slider']
grid[3,2] = widgets.HTML('<b>Initial [C m0]</b>')
grid[3,3] = elements['m0_C_slider']
grid[3,4] = widgets.HTML('<b>Initial [C m1]</b>')
grid[3,5] = elements['m1_C_slider']

grid[4,0] = widgets.HTML('<b>Initial [D m0]</b>')
grid[4,1] = elements['m0_D_slider']
grid[4,2] = widgets.HTML('<b>Initial [D m1]</b>')
grid[4,3] = elements['m1_D_slider']
grid[4,4] = widgets.HTML('<b>R1 Flux</b>')
grid[4,5] = elements['R1_slider']

grid[5,0] = widgets.HTML('<b>R2 Rate Constant</b>')
grid[5,1] = elements['K_R2_slider']
grid[5,2] = widgets.HTML('<b>R3 Rate Constant</b>')
grid[5,3] = elements['K_R3_slider']
grid[5,4] = widgets.HTML('<b>R3_b Rate Constant </b>')
grid[5,5] = elements['K_R3_b_slider']

grid[6,0] = widgets.HTML('<b>R4 Rate Constant</b>')
grid[6,1] = elements['K_R4_slider']
grid[6,2] = widgets.HTML('<b>R4_b Rate Constant </b>')
grid[6,3] = elements['K_R4_b_slider']
grid[6,4] = widgets.HTML('<b>R5 Rate Constant</b>')
grid[6,5] = elements['K_R5_slider']

grid[7,0] = widgets.HTML('<b>R1 00 Fraction</b>')
grid[7,1] = elements['f_m00_R1_slider']
grid[7,2] = widgets.HTML('<b>R1 10 Fraction</b>')
grid[7,3] = elements['f_m10_R1_slider']
grid[7,4] = widgets.HTML('<b>R1 01 Fraction</b>')
grid[7,5] = elements['f_m01_R1_slider']

grid[8,0] = widgets.HTML('<b>R1 11 Fraction</b>')
grid[8,1] = elements['f_m11_R1_slider']
grid[8,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss the results of these three simulations with your group. What is the relationship between the parameters, fluxes, and labeling patterns you see? How do the three situations we asked you to record data for differ qualitatively?
</div>

# **(2.0)** Modeling this system using FBA

We are going to treat the results of the above exercise as our ground-truth against which we compare our FBA and MFA results. So, for the sake of this exercise, our model is a 100% accurate description of our system and we have accurate parameter estimates. Even this simple metabolic model with only four metabolites had quite a few parameters though; what if we wanted to generate a steady-state flux map for our system without all of this information? Flux Balance Analysis is one approach that's used. Let's see how accurate our FBA results end up being in this extremely simple network.

<div class="alert alert-block alert-info">

**Instructions:** Run the two cells below to import the necessary cobrapy package and setup our model.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are importing the Python packages we'll need for our FBA exercises.
#-----------------------------------------------------------------------------------------------------------------------

import cobra as cb                                        # Imports the cobrapy package to handle FBA modeling
from cobra.sampling import sample                         # Imports the sampling function from cobra
from cobra.flux_analysis import flux_variability_analysis # Imports the flux variability analysis function from cobra

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are creating our FBA model to mirror the architecture we used in the kinetic exercise
#-----------------------------------------------------------------------------------------------------------------------

model = cb.Model()        # We create an empty COBRA model

A = cb.Metabolite('A')    # We then create the metabolites in our network
B = cb.Metabolite('B')
C = cb.Metabolite('C')
D = cb.Metabolite('D')

#-----------------------------------------------------------------------------------------------------------------------
# We have to create and then define the stoichiometry of all of our reactions
#-----------------------------------------------------------------------------------------------------------------------

R1 = cb.Reaction('R1')    # We create reactions ...
R1.add_metabolites({      # And then populate them with the stoichiometric coefficients of involved metabolites
    A:1.0})

R2 = cb.Reaction('R2')
R2.add_metabolites({
    A:-1.0,
    B:1.0})
R3 = cb.Reaction('R3')
R3.add_metabolites({
    B:-1.0,
    C:1.0,
    D:1.0})
R3_b = cb.Reaction('R3_b')
R3_b.add_metabolites({
    B:1.0,
    C:-1.0,
    D:-1.0})
R4 = cb.Reaction('R4')
R4.add_metabolites({
    C:-1.0,
    D:1.0})
R4_b = cb.Reaction('R4_b')
R4_b.add_metabolites({
    C:1.0,
    D:-1.0})
R5 = cb.Reaction('R5')
R5.add_metabolites({
    C:-1.0})


#-----------------------------------------------------------------------------------------------------------------------
# Finally, we add our reactions to our model.
#-----------------------------------------------------------------------------------------------------------------------

model.add_reactions([R1,R2,R3,R3_b,R4,R4_b,R5])

original_model = model

FBA is typically deployed in cases where our network is *underdetermined*; that is, we have insufficient flux information available to come up with a unique flux map. Instead, there are multiple flux maps that are all consistent with the constraints provided. All of the viable flux maps that are consistent with our constraints constitute what we'll call the **solution space**. If this space is very large, we can't extract much value from it. So, FBA uses linear optimization techniques to reduce the size of this space. We do this by specifying an objective function, which is a single or a linear combination of fluxes in our network, that we would like to maximize or minimize. Any flux in the network can be a part of our objective.

We'll say that we know that the flux R1 is  10 - let's pretend this is a substrate uptake rate that we were able to measure empirically. Given that, what does our FBA-predicted flux map look like? What is our objective function?

In real FBA studies with large-scale networks, the most common choice is the maximization of biomass accumulation. If biomass accumulation has been measured and can therefore be set as a constraint, the minimization of flux is a common choice. In this case, we'll start off by trying (as much as possible in FBA) by *not* specifying an objective. One way to accomplish this is by setting our objective as the maximization/minimization of a flux we've constrained the value of. We'll choose R1.

<div class="alert alert-block alert-info">

**Instructions:** Run the three cells to generate our interface for FBA. Change the objective function to be reaction R1, change the upper and lower bounds of R1 to 10, and run the simulation to generate a flux map. Note your results and then explore the effect of the other parameters on your solution. Is there a way to get internal fluxes that match well with our kinetic simulation results?

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines our FBA function, which accepts parameters about constraints and optimization
# and passes these to the optimization function in cobra.
#----------------------------------------------------------------------------------------------------------------

def run_FBA(model, R1_ub = 10, R1_lb = 10, R2_ub = 1000, R2_lb = 0, R3_ub = 1000, R3_lb = 0, R3_b_ub = 1000, 
        R3_b_lb = 0, R4_lb = 1000, R4_ub = 0, R4_b_ub = 1000, R4_b_lb = 0, R5_ub = 1000, R5_lb = 0, objective = 'R5',
            min_max = 'max', mode_drop = False):
    
    model.reactions.R1.lower_bound = R1_lb                    # We set all of our upper and lower bounds according to the
    model.reactions.R1.upper_bound = R1_ub                    # parameters we've set
    model.reactions.R2.lower_bound = R2_lb
    model.reactions.R2.upper_bound = R2_ub
    model.reactions.R3.lower_bound = R3_lb
    model.reactions.R3.upper_bound = R3_ub
    model.reactions.R3_b.lower_bound = R3_b_lb
    model.reactions.R3_b.upper_bound = R3_b_ub
    model.reactions.R4.lower_bound = R4_lb
    model.reactions.R4.upper_bound = R4_ub
    model.reactions.R4_b.lower_bound = R4_b_lb
    model.reactions.R4_b.upper_bound = R4_b_ub
    model.reactions.R5.lower_bound = R5_lb
    model.reactions.R5.upper_bound = R5_ub
    
    model.objective = objective                               # We set our objective to whatever reaction has been chosen
    model.objective_direction = min_max                       # and set its ddirection
    
    if mode_drop == 'pFBA':                                   # Here, we allow the user to select between pFBA, FBA, FVA
        solution = cb.flux_analysis.pfba(model)               # or sampling
        print(solution.fluxes)
    elif mode_drop == 'FBA':
        solution = model.optimize()
        print(solution.fluxes)
    elif mode_drop == 'FVA':
        FVA_solution = flux_variability_analysis(model)
        print(FVA_solution)
    else:
        samples = sample(model, 100)
        print(samples)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_FBA(model, R1_ub_slider_value = 10, R1_lb_slider_value = 10, R2_ub_slider_value = 1000, R2_lb_slider_value = 0,
                            R3_ub_slider_value = 1000, R3_lb_slider_value = 0, R3_b_ub_slider_value = 1000, R3_b_lb_slider_value = 0, 
                            R4_lb_slider_value = 0, R4_ub_slider_value = 1000, R4_b_ub_slider_value = 1000, R4_b_lb_slider_value = 0,
                            R5_ub_slider_value = 10000, R5_lb_slider_value = 0, objective_value = 'R5', min_max_value = 'maximize', 
                            pfba_drop_value = 'FBA'):

    dictionary_of_elements = {'model': model,
                              'R1_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R1_ub_slider_value),
                              'R1_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R1_lb_slider_value),
                              'R2_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R2_ub_slider_value),
                              'R2_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R2_lb_slider_value),
                              'R3_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R3_ub_slider_value),
                              'R3_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R3_lb_slider_value),
                              'R3_b_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R3_b_ub_slider_value),
                              'R3_b_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R3_b_lb_slider_value),
                              'R4_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R4_ub_slider_value),
                              'R4_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R4_lb_slider_value),
                              'R4_b_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R4_b_ub_slider_value),
                              'R4_b_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R4_b_lb_slider_value),
                              'R5_ub_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R5_ub_slider_value),                              
                              'R5_lb_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = R5_lb_slider_value),
                              'objective_dropdown': widgets.Dropdown(options = [('R1', 'R1'), ('R2', 'R2'), ('R3', 'R3'),
                                                                                ('R3_b', 'R3_b'), ('R4', 'R4'), ('R4_b', 'R4_b'),
                                                                                ('R5', 'R5')], value = objective_value),
                              'min_max_dropdown': widgets.Dropdown(options = [('minimize', 'minimize'), ('maximize','maximize')],
                                                                   value = min_max_value),
                              'mode_dropdown':widgets.Dropdown(options = [('FBA', 'FBA'), ('pFBA', 'pFBA'), ('FVA', 'FVA'),
                                                                          ('Sampling', 'Sampling')], value = pfba_drop_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements


def on_button_clicked(_):                          
    with out:
        model = original_model
        clear_output()
        FBA_interactive()
        show_inline_matplotlib_plots()
        
def FBA_interactive():                             
    R1_ub = elements['R1_ub_slider'].value
    R1_lb = elements['R1_lb_slider'].value
    R2_ub = elements['R2_ub_slider'].value
    R2_lb = elements['R2_lb_slider'].value
    R3_ub = elements['R3_ub_slider'].value
    R3_lb = elements['R3_lb_slider'].value
    R3_b_ub = elements['R3_b_ub_slider'].value
    R3_b_lb = elements['R3_b_lb_slider'].value
    R4_ub = elements['R4_ub_slider'].value
    R4_lb = elements['R4_lb_slider'].value
    R4_b_ub = elements['R4_b_ub_slider'].value
    R4_b_lb = elements['R4_b_lb_slider'].value
    R5_ub = elements['R5_ub_slider'].value
    R5_lb = elements['R5_lb_slider'].value
    objective = elements['objective_dropdown'].value
    min_max = elements['min_max_dropdown'].value
    pfba = elements['mode_dropdown'].value

    run_FBA(model, R1_ub, R1_lb, R2_ub, R2_lb, R3_ub, R3_lb, R3_b_ub, R3_b_lb, R4_lb,
        R4_ub, R4_b_ub, R4_b_lb, R5_ub, R5_lb, objective, min_max, pfba)
    

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_FBA(model = model)      

elements['button'].on_click(on_button_clicked)        
                                                       
out = widgets.Output()                                

grid = widgets.GridspecLayout(9, 4)                   

grid[0,0] = widgets.HTML('<b>R1 Upper Bound</b>') 
grid[0,1] = elements['R1_ub_slider']
grid[0,2] = widgets.HTML('<b>R1 Lower Bound</b>')
grid[0,3] = elements['R1_lb_slider']

grid[1,0] = widgets.HTML('<b>R2 Upper Bound</b>') 
grid[1,1] = elements['R2_ub_slider']
grid[1,2] = widgets.HTML('<b>R2 Lower Bound</b>')
grid[1,3] = elements['R2_lb_slider']

grid[2,0] = widgets.HTML('<b>R3 Upper Bound</b>') 
grid[2,1] = elements['R3_ub_slider']
grid[2,2] = widgets.HTML('<b>R3 Lower Bound</b>')
grid[2,3] = elements['R3_lb_slider']

grid[3,0] = widgets.HTML('<b>R3_b Upper Bound</b>') 
grid[3,1] = elements['R3_b_ub_slider']
grid[3,2] = widgets.HTML('<b>R3_b Lower Bound</b>')
grid[3,3] = elements['R3_b_lb_slider']

grid[4,0] = widgets.HTML('<b>R4 Upper Bound</b>') 
grid[4,1] = elements['R4_ub_slider']
grid[4,2] = widgets.HTML('<b>R4 Lower Bound</b>')
grid[4,3] = elements['R4_lb_slider']

grid[5,0] = widgets.HTML('<b>R4_b Upper Bound</b>') 
grid[5,1] = elements['R4_b_ub_slider']
grid[5,2] = widgets.HTML('<b>R4_b Lower Bound</b>')
grid[5,3] = elements['R4_b_lb_slider']

grid[6,0] = widgets.HTML('<b>R5 Upper Bound</b>') 
grid[6,1] = elements['R5_ub_slider']
grid[6,2] = widgets.HTML('<b>R5 Lower Bound</b>')
grid[6,3] = elements['R5_lb_slider']

grid[7,0] = widgets.HTML('<b>Objective</b>')
grid[7,1] = elements['objective_dropdown']
grid[7,2] = widgets.HTML('<b>Min/Max</b>')
grid[7,3] = elements['min_max_dropdown']

grid[8,0] = widgets.HTML('<b>Mode</b>')
grid[8,1] = elements['mode_dropdown']
grid[8,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> After exploring the use of FBA with the above interactive widget, join the class for a discussion about the strengths and limitations of FBA.
</div>

# **(3.0)** FVA and Sampling

As a follow-up on the flux map we just generated, is this one solution unique? That is, what does the solution space look like? When we run an optimization like this, we get a single solution within the space, not a characterization of the entire space. There are two approaches for characterizing the solution space:

1. Flux Variability Analysis
2. Random Sampling

We will use both below.

## **(3.1)** Flux Variability Analysis

Flux Variability Analysis (FVA) involves doing a series of optimizations to figure out, given optimization of our objective function, what the upper and lower bounds are on each of our reactions. This can be done for all reactions in our model or for just a subset. 

<div class="alert alert-block alert-info">

**Instructions:** Run the cell below to generate our interface for FBA. Use the "mode" dropdown menu to perform FVA.

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_FBA(model = original_model)      

elements['button'].on_click(on_button_clicked)         

out = widgets.Output()                                 

grid = widgets.GridspecLayout(9,4)                    

grid[0,0] = widgets.HTML('<b>R1 Upper Bound</b>') 
grid[0,1] = elements['R1_ub_slider']
grid[0,2] = widgets.HTML('<b>R1 Lower Bound</b>')
grid[0,3] = elements['R1_lb_slider']

grid[1,0] = widgets.HTML('<b>R2 Upper Bound</b>') 
grid[1,1] = elements['R2_ub_slider']
grid[1,2] = widgets.HTML('<b>R2 Lower Bound</b>')
grid[1,3] = elements['R2_lb_slider']

grid[2,0] = widgets.HTML('<b>R3 Upper Bound</b>') 
grid[2,1] = elements['R3_ub_slider']
grid[2,2] = widgets.HTML('<b>R3 Lower Bound</b>')
grid[2,3] = elements['R3_lb_slider']

grid[3,0] = widgets.HTML('<b>R3_b Upper Bound</b>') 
grid[3,1] = elements['R3_b_ub_slider']
grid[3,2] = widgets.HTML('<b>R3_b Lower Bound</b>')
grid[3,3] = elements['R3_b_lb_slider']

grid[4,0] = widgets.HTML('<b>R4 Upper Bound</b>') 
grid[4,1] = elements['R4_ub_slider']
grid[4,2] = widgets.HTML('<b>R4 Lower Bound</b>')
grid[4,3] = elements['R4_lb_slider']

grid[5,0] = widgets.HTML('<b>R4_b Upper Bound</b>') 
grid[5,1] = elements['R4_b_ub_slider']
grid[5,2] = widgets.HTML('<b>R4_b Lower Bound</b>')
grid[5,3] = elements['R4_b_lb_slider']

grid[6,0] = widgets.HTML('<b>R5 Upper Bound</b>') 
grid[6,1] = elements['R5_ub_slider']
grid[6,2] = widgets.HTML('<b>R5 Lower Bound</b>')
grid[6,3] = elements['R5_lb_slider']

grid[7,0] = widgets.HTML('<b>Objective</b>')
grid[7,1] = elements['objective_dropdown']
grid[7,2] = widgets.HTML('<b>Min/Max</b>')
grid[7,3] = elements['min_max_dropdown']

grid[8,0] = widgets.HTML('Mode')
grid[8,1] = elements['mode_dropdown']
grid[8,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss these results with your group. What reactions have large flux ranges associated with them? How do we interpret these results? After discussing within your group, join the class for a whole class discussion.
</div>

## **(3.2)** Random Sampling

To get examples of actual flux maps from within our solution space, rather than just max/min values for each reaction, we can use a sampling procedure to get a bunch of random flux maps from it. Let's try this now.

<div class="alert alert-block alert-info">

**Instructions:** Run the cell below to generate our interface for FBA. Use the "mode" dropdown menu to perform sampling.

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_FBA(model=model)        

elements['button'].on_click(on_button_clicked)         

out = widgets.Output()                                 

grid = widgets.GridspecLayout(9,4)                    

grid[0,0] = widgets.HTML('<b>R1 Upper Bound</b>') 
grid[0,1] = elements['R1_ub_slider']
grid[0,2] = widgets.HTML('<b>R1 Lower Bound</b>')
grid[0,3] = elements['R1_lb_slider']

grid[1,0] = widgets.HTML('<b>R2 Upper Bound</b>') 
grid[1,1] = elements['R2_ub_slider']
grid[1,2] = widgets.HTML('<b>R2 Lower Bound</b>')
grid[1,3] = elements['R2_lb_slider']

grid[2,0] = widgets.HTML('<b>R3 Upper Bound</b>') 
grid[2,1] = elements['R3_ub_slider']
grid[2,2] = widgets.HTML('<b>R3 Lower Bound</b>')
grid[2,3] = elements['R3_lb_slider']

grid[3,0] = widgets.HTML('<b>R3_b Upper Bound</b>') 
grid[3,1] = elements['R3_b_ub_slider']
grid[3,2] = widgets.HTML('<b>R3_b Lower Bound</b>')
grid[3,3] = elements['R3_b_lb_slider']

grid[4,0] = widgets.HTML('<b>R4 Upper Bound</b>') 
grid[4,1] = elements['R4_ub_slider']
grid[4,2] = widgets.HTML('<b>R4 Lower Bound</b>')
grid[4,3] = elements['R4_lb_slider']

grid[5,0] = widgets.HTML('<b>R4_b Upper Bound</b>') 
grid[5,1] = elements['R4_b_ub_slider']
grid[5,2] = widgets.HTML('<b>R4_b Lower Bound</b>')
grid[5,3] = elements['R4_b_lb_slider']

grid[6,0] = widgets.HTML('<b>R5 Upper Bound</b>') 
grid[6,1] = elements['R5_ub_slider']
grid[6,2] = widgets.HTML('<b>R5 Lower Bound</b>')
grid[6,3] = elements['R5_lb_slider']

grid[7,0] = widgets.HTML('<b>Objective</b>')
grid[7,1] = elements['objective_dropdown']
grid[7,2] = widgets.HTML('<b>Min/Max</b>')
grid[7,3] = elements['min_max_dropdown']

grid[8,0] = widgets.HTML('Mode')
grid[8,1] = elements['mode_dropdown']
grid[8,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss these results with your group. How do these results differ from what we were seeing above with FVA?
</div>

In publications using FBA and these kinds of sampling strategies, you will see histograms of the flux values taken by particular reactions across large numbers of flux maps. Let's look at an example from our toy model:

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell performs the sampling procedure and then plots out the distributions of our fluxes
#----------------------------------------------------------------------------------------------------------------

samples = sample(model, 1000)                      # Generates 1000 random flux maps consistent with our previously
                                                   # defined constraints

fig, ax = plt.subplots(3, 3, figsize=(10, 10))     # We initialize our series of plots

ax[0,0].hist(samples['R1'])                        # Then we plot a histogram of each reaction in its own plot
ax[0,0].set_title('R1', fontweight = 'bold')
ax[0,0].set_xlabel('Flux', fontweight = 'bold')
ax[0,0].set_ylabel('Count', fontweight = 'bold')

ax[0,1].hist(samples['R2'])
ax[0,1].set_title('R2', fontweight = 'bold')
ax[0,1].set_xlabel('Flux', fontweight = 'bold')
ax[0,1].set_ylabel('Count', fontweight = 'bold')

ax[0,2].hist(samples['R3'])
ax[0,2].set_title('R3', fontweight = 'bold')
ax[0,2].set_xlabel('Flux', fontweight = 'bold')
ax[0,2].set_ylabel('Count', fontweight = 'bold')

ax[1,0].hist(samples['R3_b'])
ax[1,0].set_title('R3_b', fontweight = 'bold')
ax[1,0].set_xlabel('Flux', fontweight = 'bold')
ax[1,0].set_ylabel('Count', fontweight = 'bold')

ax[1,1].hist(samples['R4'])
ax[1,1].set_title('R4', fontweight = 'bold')
ax[1,1].set_xlabel('Flux', fontweight = 'bold')
ax[1,1].set_ylabel('Count', fontweight = 'bold')

ax[1,2].hist(samples['R4_b'])
ax[1,2].set_title('R4_b', fontweight = 'bold')
ax[1,2].set_xlabel('Flux', fontweight = 'bold')
ax[1,2].set_ylabel('Count', fontweight = 'bold')

ax[2,0].hist(samples['R5'])
ax[2,0].set_title('R5', fontweight = 'bold')
ax[2,0].set_xlabel('Flux', fontweight = 'bold')
ax[2,0].set_ylabel('Count', fontweight = 'bold')

plt.tight_layout()

## **(3.3)** A More Practical FBA Example

<div class="alert alert-block alert-info">

**Note:** The metabolic models used below were downloaded from the BiGG models resource. Citations for the BiGG models resource and the original publications that the models were presented in can be found below:
    
**BiGG Models**: Z. A. King, et al., BiGG Models: A platform for integrating, standardizing and sharing genome-scale models. Nucleic Acids Res. 44, D515–D522 (2016).
    
**iJO1366**: J. D. Orth, et al., A comprehensive genome-scale reconstruction of Escherichia coli metabolism—2011. Mol. Syst. Biol. 7, 535 (2011).
    
**E.coli core model**: J. D. Orth, et al., Reconstruction and Use of Microbial Metabolic Networks: the Core Escherichia coli Metabolic Model as an Educational Guide. EcoSal Plus 4 (2010).

</div>

Now, FBA is not typically used for these kinds of extremely small networks. Rather, it is most often used with genome-scale models that attempt to capture all of the biochemical reactions possible in an organism given its genome annotation. Let's take a look at a network of *E. coli.*

<div class="alert alert-block alert-info">

**Instructions:** Look at the Excel spreadsheet titled **iJO1366.xls** to examine the E. coli metabolic network. Then, turn your attention to the instructors for a walkthrough of the important elements in an SBML format model.

</div>

As we have seen seen from earlier examples, FBA can struggle with predicting internal fluxes without extensive flux constraints. But, a common demonstration of its power as a tool for accounting for the movement and matter and energy through a a metabolic network is its ability to predict biomass accumulation rates in microbial systems. Here is an example:

<div class="alert alert-block alert-info">

**Instructions:** Run the cells below to load in the model, set the upper and lower bounds on glucose uptake to 10, and run the simulation.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are loading up a genome scale model of E coli and using it
#-----------------------------------------------------------------------------------------------------------------------

ecoli_model = cb.io.read_sbml_model('iJO1366.xml')           # First we load in the genome scale model

ecoli_model.reactions.EX_glc__D_e.upper_bound = -10          # Then we constrain the upper and lower bounds on glucose uptake
ecoli_model.reactions.EX_glc__D_e.lower_bound = -10

ecoli_model.objective = 'BIOMASS_Ec_iJO1366_WT_53p95M'       # We set our objective function and specify we want to maximize
ecoli_model.objective_direction = 'maximize'

solution = cb.flux_analysis.pfba(ecoli_model)                # Finally, we run the optimization using pFBA
print('Our objective function value is: ', solution['BIOMASS_Ec_iJO1366_WT_53p95M'])

What does the flux map look like when we perform this optimization? We can save the flux map to a .csv file and take a look.

<div class="alert alert-block alert-info">

**Instructions:** Run the code cell below and examine the .csv file that's generated.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# This cell saves the results of the previous optimization
#-----------------------------------------------------------------------------------------------------------------------

solution_df = pd.DataFrame(solution.fluxes)
solution_df.to_csv('Ecoli_Flux_Map.csv')

Note that this is **one** flux map that's consistent with the optimization of biomass accumulation. What about the variability in the solution space? FVA on any appreciable set of the reactions in this network would be very computationally expensive, so we can insatead perform FVA for a small set of important reactions. Below, we look at citrate synthase (TCA cycle), phosphofructokinase (glycolysis), and ATP synthase.

<div class="alert alert-block alert-info">

**Instructions:** Find the reaction names for citrate synthase, hexokinase, and ATP synthase in the *E.coli* model, then plug those names into the code cell below to plot out their distributions from the sampling of the solution space we perform.

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# Here, we designate the reactions we're interested in and then perform FVA on them
#----------------------------------------------------------------------------------------------------------------

CitrateSynthase = ecoli_model.reactions. ### Put the name of the citrate synthase reaction here, after the period
Phosphofructokinase = ecoli_model.reactions. ### Put the name of the phosphofructokinase reaction here
ATPSynth = ecoli_model.reactions. ### Put the name of the ATP synthase reaction here

reactions = [CitrateSynthase, Phosphofructokinase, ATPSynth]

FVA_solution = flux_variability_analysis(ecoli_model,reactions)

FVA_solution

It looks like of these three, phosphofructokinase is the one that shows pretty wide variability, whereas the others are uniquely defined. A good next step to digging into the meaning of this would be to perform sampling. Unfortunately, sampling procedures scale extremely poorly with model size. We'd be here all day if we waited even for a relatively small number of samples with a model of this size. Instead, let's see if we observe this same behavior in a much smaller model of E. coli's core metabolism ...

<div class="alert alert-block alert-info">

**Instructions:** Run the cells below to load in the *E coli* core model and perform a sampling analysis.

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# First, we find what our optimal biomass is in this reduced version of the Ecoli network
#----------------------------------------------------------------------------------------------------------------

ecoli_core_model = cb.io.read_sbml_model('e_coli_core.xml')   # This time, loading in the core model
ecoli_core_model.reactions.EX_glc__D_e.upper_bound = -10      # Setting our bounds, just the same as last time
ecoli_core_model.reactions.EX_glc__D_e.lower_bound = -10
ecoli_core_model.objective = 'BIOMASS_Ecoli_core_w_GAM'       # Setting our objective function ...
ecoli_core_model.objective_direction = 'maximize'             # And direction ...

solution = ecoli_core_model.optimize()                        # And optimizing
solution

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# With the optimal biomass above, we can constrain the upper and lower bounds of the biomass reaction to that
# value and then perform our sampling procedure
#----------------------------------------------------------------------------------------------------------------+

ecoli_core_model.reactions.BIOMASS_Ecoli_core_w_GAM.upper_bound = 0.874
ecoli_core_model.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = 0.873

samples = sample(ecoli_core_model,1000)

plt.hist(samples['PFK'])

As you can see, this isn't the same as the range that we saw in the full-scale model. But this makes sense because the core model leaves out a lot of metabolic reactions that  provided the flexibility needed to achieve optimal biomass accumulation without using phosphofructokinase. In fact, we can check the number of reactions that are present in the two models.

In [ ]:
print('The number of reactions in the full model: ',len(ecoli_model.reactions))
print('The number of reactions in the core model: ',len(ecoli_core_model.reactions))

Here we have a tradeoff between the interpretability and computational convenience of the core model versus the detail provided by the full genome scale model. This is something to pay close attention to if you intend to do any work in FBA yourself.

<div class="alert alert-block alert-warning"> 
    <b>Break Time</b> Once all groups have finished discussing their findings, we'll take a fifteen minute break.

## **(4.0)** Metabolic Flux Analysis

<div class="alert alert-info">
    <b>Note:</b> The code used to run the 13C-MFA analysis in this section is adapted from the mfapy demonstration code developed by Fumio Matsuda for the following publication: <b> F. Matsuda, et al., mfapy: An open-source Python package for 13C-based metabolic flux analysis. Metab. Eng. Commun. 13, e00177 (2021). </b> For the original GitHub repository that accompanies the mfapy publication, go to: <b> https://github.com/fumiomatsuda/mfapy </b>
</div>

Now let's move onto Metabolic Flux Analysis (MFA). Note that there are multiple forms of MFA. In this notebook, we will be covering stationary 13C-MFA, which incorporates isotopic labeling data to come up with accurate estimates of intracellular fluxes and assumes the system is at both metabolic **and** isotopic steady state. We'll be using the package *mfapy* for these demonstrations. 

What we're going to do is use MFA to try to calculate the fluxes we got earlier in our kinetic network using:
1. The model architecture we used
2. The labeling data of all the metabolites

To do this, we'll need to provide:

1. A description of all the reactions in our model, including the positions of all labeled carbons in substrates and products.
2. A description of the constraints on our problem (i.e. which fluxes are fixed and to what values, and what the upper and lower bounds are on all reactions)
3. A description of all our metabolites, including how many labeled positions there are.

We will provide the first two things, but will ask you to enter the values you got from your kinetic simulation for the third. 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Pause here for a walkthrough of the contents of the model and status files. 
</div>

<div class="alert alert-block alert-info">

**Instructions:** Run the cells below to load in the MFA model architecture and constraints. 
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# Here, we import the necessary package, load up our model, and set our constraints. We also set our labeled substrate information
#----------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  # This line loads in our model elements
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                                 # This line sets our model constraints according to them
model.update()                                                                                                                                   # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                                     # This line creates a carbon source object
cs.set_each_isotopomer('Aex',{'#00':0.0,'#01':1.0,'#11':0.0})                                                                                    # This line populates that object with a 100% m1 labeled 
                                                                                                                                                 # carbon source

Now that the network and constraints are in place, we can input labeling data. To do this, we need to enter the M0, M1, and M2 fractional abundances for all of the metabolites in the network into the provided template file "MDV_Template.txt". 

<div class="alert alert-block alert-info">

**Instructions:** Populate the MDV_Template.txt file with the steady-state labeling levels you got from the three sets of parameters in the kinetic modeling exercise at the beginning of this notebook. Save the results from the first set of parameters we tested to a file named "MDV_Case1.txt", the results from the second set of parameters to a file named "MDV_Case2.txt", and the results of the third set of parameters to a file named "MDV_Case3.txt"
</div>

Now that these datasets have been set up, we can carry out 13C-MFA. We will start with the case where we are providing unlabeled substrate.

<div class="alert alert-block alert-info">

**Instructions:** Run the cells below to carry out the MFA procedure for the first case. Note that this code may take a bit of time to run.
</div>

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  # This line loads in our model elements.
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                                 # This line sets our model constraints according to them
model.update()                                                                                                                                   # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                                     # This line creates a carbon source object

mdv_observed = model.load_mdv_data('MDV_Case1.txt')                                                                                              # This line loads in our labeling information 
mdv_observed.set_std(0.001, method = 'absolute')                                                                                                 # This line sets the standard deviation of our measurements
mdv_observed.add_gaussian_noise(0.001)                                                                                                            # This line adds a small amount of random noise to the data
cs.set_each_isotopomer('Aex',{'#00':1.0,'#01':0.0,'#11':0.0})                                                                                    # This line sets our carbon source information
model.set_experiment('ex1',mdv_observed,cs)                                                                                                      # This line defines and sets an "experiment" based on 
                                                                                                                                                 # our carbon source and labeling info
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate many random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            # Generating initial states
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    # Adding initial state to results
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):                                                                                      # Beginning the loop
    for method in ["SLSQP","LN_SBPLX"]:                                                                  # Each loop we take the best fit we've had so far and use it to start
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case1.csv'

state, flux_opt1 = model.generate_initial_states(100, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)
#
# Data store
#
record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)
 
    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Let's discuss these results as a whole class.
</div>

Now we look at Case 2.

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  # This line loads in our model elements.
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                                 # This line sets our model constraints according to them
model.update()                                                                                                                                   # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                                     # This line creates a carbon source object

mdv_observed = model.load_mdv_data('MDV_Case2.txt')                                                                                              # This line loads in our labeling information 
mdv_observed.set_std(0.01, method = 'absolute')                                                                                                  # This line sets the standard deviation of our measurements
#mdv_observed.add_gaussian_noise(0.01)                                                                                                           # This line adds a small amount of random noise to the data
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#01':0.0,'#11':0.0})                                                                          # This line sets our carbon source information
model.set_experiment('ex1',mdv_observed,cs)                                                                                                      # This line defines and sets an "experiment" based on 
                                                                                                                                                 # our carbon source and labeling info
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a number of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            # Generating initial states
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    # Adding initial state to results
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  # Each loop we take the best fit found so far and use it to start
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case2.csv'

state, flux_opt1 = model.generate_initial_states(100, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)
#
# Data store
#
record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

In [ ]:
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)


<div class="alert alert-block alert-success">
    <b>Discussion:</b> Let's discuss these results as a whole class.
</div>

Finally, we look at the final dataset.

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  # This line loads in our model elements.
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                                 # This line sets our model constraints according to them
model.update()                                                                                                                                   # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                                     # This line creates a carbon source object

mdv_observed = model.load_mdv_data('MDV_Case3.txt')                                                                                              # This line loads in our labeling information 
mdv_observed.set_std(0.01, method = 'absolute')                                                                                                  # This line sets the standard deviation of our measurements
#mdv_observed.add_gaussian_noise(0.01)                                                                                                           # This line adds a small amount of random noise to the data
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#11':0.0})                                                                                    # This line sets our carbon source information
model.set_experiment('ex1',mdv_observed,cs)                                                                                                      # This line defines and sets an "experiment" based on 
                                                                                                                                                 # our carbon source and labeling info
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a bunch of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            # Generating initial states
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    # Adding initial state to results
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  # Each loop we take the best fit we've had so far and use it to start
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case3.csv'

state, flux_opt1 = model.generate_initial_states(100, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)
#
# Data store
#
record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> After getting your results, let's come back together for another whole class discussion.
</div>

## **(4.1)** Model Selection

Note before that when we were discussing the inputs for the MFA modeling exercise, we emphasized that we had to provide an **assumed** model architecture. What happens if we make an incorrect assumption and leave something out that's important?

Below, we introduce a new reaction that allows a small amount of unlabeled (m0) C into the system, as you can see in the updated network diagram. What happens when we generate labeling data using this new model and then try to fit it using MFA and our old model architecture?

![alt text](NetworkDiagram_2.png "Title")

<div class="alert alert-block alert-info">

**Instructions:** Run the cells below to run the kinetic simulation. Record the fluxes and fractional labeling values.
</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation, as well as the function that carries
# out the simulation itself. 
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_dCdt_f(R3, R4, R5, R6, f_m00_B, f_m10_B, f_m01_B, f_m11_B, f_m0_C, f_m1_C, f_m0_R6, f_m1_R6):   # This function has been modified to account for the influx through R6
    
    dm0dt = (R6 * f_m0_R6) + (R3 * f_m00_B) + (R3 * f_m10_B) - (R4 * f_m0_C) - (R5 * f_m0_C)
    dm1dt = (R6 * f_m1_R6) + (R3 * f_m01_B) + (R3 * f_m11_B) - (R4 * f_m1_C) - (R5 * f_m1_C)
    
    return dm0dt, dm1dt

def kinetic_simulation_altered(total_time = 1000, timestep = 1, m00_A = 0, m10_A = 0, m01_A = 0, m11_A = 0, m00_B = 0, m10_B = 0, m01_B = 0, m11_B = 0, m0_C = 0, m1_C = 0, m0_D = 0, m1_D = 0,  
        R1 = 10, K_R2 = 0.1, K_R3 = 0.1, K_R3_b = 0.01, K_R4 = 0.1, K_R4_b = 0.01, K_R5 = 0.1, R6 = 1, f_m00_R1 = 0, f_m10_R1 = 1, f_m01_R1 = 0, f_m11_R1 = 0, f_m0_R6 = 0, f_m1_R6 = 1):
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using reversible first-order kinetics.
    Accepts arguments to both export generated data and import another group's data to 
    compare with simulation results. Can also subset generated data and add random noise
    before exporting.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    
    m00_A = A numeric value representing the initial concentration of m00 A
    
    m10_A = A numeric value representing the initial concentration of m10 A
    
    m01_A = A numeric value representing the initial concentration of m01 A
    
    m11_A = A numeric value representing the initial concentration of m11 A
    
    m00_B = A numeric value representing the initial concentration of m00 B
    
    m10_B = A numeric value representing the initial concentration of m10 B
    
    m01_B = A numeric value representing the initial concentration of m01 B
    
    m11_B = A numeric value representing the initial concentration of m11 B
    
    m0_C = A numeric value representing the initial concentration of m0 C
    
    m1_C = A numeric value representing the initial concentration of m1 C
    
    m0_D = A numeric value representing the initial concentration of m0 D
    
    m1_D = A numeric value representing the initial concentration of m1 D   
    
    R1 = A numeric value representing the flux through reaction R1
    
    K_R2 = A numeric value representing the first-order rate constant for the
    reaction R2
    
    K_R3 = A numeric value representing the first-order rate constant for the
    reaction R3
    
    K_R3_b = A numeric value representing the first-order rate constant for the
    reaction R3_b
    
    K_R4 = A numeric value representing the first-order rate constant for the
    reaction R4
    
    K_R4_b = A numeric value representing the first-order rate constant for the
    reaction R4_b
    
    K_R5 = A numeric value representing the first-order rate constant for the
    reaction R5
    
    K_R6 = A numeric value representing the flux through reaction R6

    f_m0_R1 = A numeric value representing the fraction of R1 that is m0
    
    f_m1_R1 = A numeric value representing the fraction of R1 that is m1
    
    f_m2_R1 = A numeric value representing the fraction of R1 that is m2
    
    f_m00_R6 = A numeric value representing the fraction of R6 hat is m00
    
    f_m10_R6 = A numeric value representing the fraction of R6 that is m10
    
    f_m01_R6 = A numeric value representing the fraction of R6 that is m11
    
    OUTPUTS
    
    None
    
    '''
    
    times = np.arange(0, total_time, timestep)                    
                                                                    
    A_total = m00_A + m10_A + m01_A + m11_A                      
    B_total = m00_B + m10_B + m01_B + m11_B                      
    C_total = m0_C + m1_C
    D_total = m0_C + m1_C
    
    m00_A_list = [m00_A]                                                                                                      
    m00_B_list = [m00_B]                                                                                                  
    m0_C_list = [m0_C]                                                    
    m0_D_list = [m0_D]                                                   
                                                                          
    m10_A_list = [m10_A]                                                                                                   
    m10_B_list = [m10_B]                                                                                                  
    m1_C_list = [m1_C]                                                   
    m1_D_list = [m1_D]  
    
    m01_A_list = [m01_A]                                                                                                        
    m01_B_list = [m01_B]                                                                                                  
                                                                            
    m11_A_list = [m11_A]
    m11_B_list = [m11_B]
        
    A_total_list = [A_total]                                     
    B_total_list = [B_total]
    C_total_list = [C_total]
    D_total_list = [D_total]
        
    f_m00_A, f_m10_A, f_m01_A, f_m11_A = evaluate_fraction_B(m00_A, m10_A, m01_A, m11_A)
    f_m00_B, f_m10_B, f_m01_B, f_m11_B = evaluate_fraction_B(m00_B, m10_B, m01_B, m11_B)
    f_m0_C, f_m1_C = evaluate_fraction(m0_C, m1_C)
    f_m0_D, f_m1_D = evaluate_fraction(m0_D, m1_D)
    
    f_m00_A_list = [f_m00_A]
    f_m10_A_list = [f_m10_A]
    f_m01_A_list = [f_m01_A]
    f_m11_A_list = [f_m11_A]
    f_m00_B_list = [f_m00_B]
    f_m10_B_list = [f_m10_B]
    f_m01_B_list = [f_m01_B]
    f_m11_B_list = [f_m11_B]
    f_m0_C_list = [f_m0_C]
    f_m1_C_list = [f_m1_C]
    f_m0_D_list = [f_m0_D]
    f_m1_D_list = [f_m1_D]  
    f_m00_R1_list = [f_m00_R1]
    f_m10_R1_list = [f_m10_R1]
    f_m01_R1_list = [f_m01_R1]
    f_m11_R1_list = [f_m11_R1]
    f_m0_R6_list = [f_m0_R6]
    f_m1_R6_list = [f_m1_R6]
    
    f_m1_A_list = [f_m10_A + f_m01_A]
    f_m1_B_list = [f_m10_B + f_m01_B]
    
    R1_list = [R1]            
    R2_list = [evaluate_flux(K_R2, A_total_list[-1])]
    R3_list = [evaluate_flux(K_R3, B_total_list[-1])]
    R3_b_list = [evaluate_flux_bimolecular(K_R3_b, C_total_list[-1], D_total_list[-1])]
    R4_list = [evaluate_flux(K_R4, C_total_list[-1])]
    R4_b_list = [evaluate_flux(K_R4_b, D_total_list[-1])]
    R5_list = [evaluate_flux(K_R5, C_total_list[-1])]
    R6_list = [R6]
    
    for i in times:                                              

        current_R1 = R1            
        current_R2 = evaluate_flux(K_R2, A_total_list[-1])
        current_R3 = evaluate_flux(K_R3, B_total_list[-1])
        current_R3_b = evaluate_flux_bimolecular(K_R3_b, C_total_list[-1], D_total_list[-1])  
        current_R4 = evaluate_flux(K_R4, C_total_list[-1])                                    
        current_R4_b = evaluate_flux(K_R4_b, D_total_list[-1])
        current_R5 = evaluate_flux(K_R5,C_total_list[-1])
        current_R6 = R6
        
        dm00_A, dm10_A, dm01_A, dm11_A = evaluate_dAdt_f(current_R1, current_R2, f_m00_R1_list[-1], f_m10_R1_list[-1], f_m01_R1_list[-1], f_m11_R1_list[-1], f_m00_A_list[-1], f_m10_A_list[-1], f_m01_A_list[-1], f_m11_A_list[-1])
        dm00_B, dm10_B, dm01_B, dm11_B = evaluate_dBdt_f(current_R2, current_R3, f_m00_A_list[-1], f_m10_A_list[-1], f_m01_A_list[-1], f_m11_A_list[-1], f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1])
        dm0_C, dm1_C = evaluate_dCdt_f(current_R3, current_R4, current_R5, current_R6, f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1], f_m0_C_list[-1], f_m1_C_list[-1], f_m0_R6_list[-1], f_m1_R6_list[-1])
        dm0_D, dm1_D = evaluate_dDdt_f(current_R3, current_R4, f_m00_B_list[-1], f_m10_B_list[-1], f_m01_B_list[-1], f_m11_B_list[-1], f_m0_C_list[-1], f_m1_C_list[-1])                                     
        
        current_m00_A = m00_A_list[-1] + dm00_A * timestep
        current_m10_A = m10_A_list[-1] + dm10_A * timestep
        current_m01_A = m01_A_list[-1] + dm01_A * timestep
        current_m11_A = m11_A_list[-1] + dm11_A * timestep

        current_m00_B = m00_B_list[-1] + dm00_B * timestep
        current_m10_B = m10_B_list[-1] + dm10_B * timestep
        current_m01_B = m01_B_list[-1] + dm01_B * timestep
        current_m11_B = m11_B_list[-1] + dm11_B * timestep

        current_m0_C = m0_C_list[-1] + dm0_C * timestep
        current_m1_C = m1_C_list[-1] + dm1_C * timestep
        current_m0_D = m0_D_list[-1] + dm0_D * timestep
        current_m1_D = m1_D_list[-1] + dm1_D * timestep
        
        current_A_total = current_m00_A + current_m10_A + current_m01_A + current_m11_A
        current_B_total = current_m00_B + current_m10_B + current_m01_B + current_m11_A
        current_C_total = current_m0_C + current_m1_C
        current_D_total = current_m0_D + current_m1_D
        
        current_f_m00_A, current_f_m10_A, current_f_m01_A, current_f_m11_A = evaluate_fraction_B(current_m00_A, current_m10_A, current_m01_A, current_m11_A)
        current_f_m00_B, current_f_m10_B, current_f_m01_B, current_f_m11_B = evaluate_fraction_B(current_m00_B, current_m10_B, current_m01_B, current_m11_B)
        current_f_m0_C, current_f_m1_C = evaluate_fraction(current_m0_C, current_m1_C)
        current_f_m0_D, current_f_m1_D = evaluate_fraction(current_m0_D, current_m1_D)
        current_f_m00_R1 = f_m00_R1
        current_f_m10_R1 = f_m10_R1
        current_f_m01_R1 = f_m01_R1
        current_f_m11_R1 = f_m11_R1
        current_f_m0_R6 = f_m0_R6
        current_f_m1_R6 = f_m1_R6
                        
        dm00_B, dm10_B, dm01_B, dm11_B = evaluate_dBdt_b(current_R3_b, current_f_m0_D, current_f_m1_D, current_f_m0_C, current_f_m1_C)
        dm0_C, dm1_C = evaluate_dCdt_b(current_R3_b, current_R4_b, current_f_m0_C, current_f_m1_C, current_f_m0_D, current_f_m1_D)
        dm0_D, dm1_D = evaluate_dDdt_b(current_R3_b, current_R4_b, current_f_m0_D, current_f_m1_D)

        current_m00_A = current_m00_A
        current_m10_A = current_m10_A
        current_m01_A = current_m01_A
        current_m11_A = current_m11_A
        current_m00_B = current_m00_B + dm00_B * timestep
        current_m10_B = current_m10_B + dm10_B * timestep
        current_m01_B = current_m01_B + dm01_B * timestep
        current_m11_B = current_m11_B + dm11_B * timestep
        current_m0_C = current_m0_C + dm0_C * timestep
        current_m1_C = current_m1_C + dm1_C * timestep
        current_m0_D = current_m0_D + dm0_D * timestep
        current_m1_D = current_m1_D + dm1_D * timestep
                                       
        current_A_total = current_m00_A + current_m10_A + current_m01_A + current_m11_A
        current_B_total = current_m00_B + current_m10_B + current_m01_B + current_m11_A
        current_C_total = current_m0_C + current_m1_C
        current_D_total = current_m0_D + current_m1_D          
                
        current_f_m00_A, current_f_m10_A, current_f_m01_A, current_f_m11_A = evaluate_fraction_B(current_m00_A, current_m10_A, current_m01_A, current_m11_A)
        current_f_m00_B, current_f_m10_B, current_f_m01_B, current_f_m11_B = evaluate_fraction_B(current_m00_B, current_m10_B, current_m01_B, current_m11_B)
        current_f_m0_C, current_f_m1_C = evaluate_fraction(current_m0_C, current_m1_C)
        current_f_m0_D, current_f_m1_D = evaluate_fraction(current_m0_D, current_m1_D)
        current_f_m00_R1 = f_m00_R1
        current_f_m10_R1 = f_m10_R1
        current_f_m01_R1 = f_m01_R1
        current_f_m11_R1 = f_m11_R1
        current_f_m0_R6 = f_m0_R6
        current_f_m1_R6 = f_m1_R6
        
        m00_A_list.append(current_m00_A)
        m10_A_list.append(current_m10_A)
        m01_A_list.append(current_m01_A)
        m11_A_list.append(current_m11_A)
        m00_B_list.append(current_m00_B)
        m10_B_list.append(current_m10_B)
        m01_B_list.append(current_m01_B)
        m11_B_list.append(current_m11_B)
        m0_C_list.append(current_m0_C)
        m1_C_list.append(current_m1_C)
        m0_D_list.append(current_m0_D)
        m1_D_list.append(current_m1_D)
                                
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        C_total_list.append(current_C_total)
        D_total_list.append(current_D_total)
          
        f_m00_A_list.append(current_f_m00_A)
        f_m10_A_list.append(current_f_m10_A)
        f_m01_A_list.append(current_f_m01_A)
        f_m11_A_list.append(current_f_m11_A)

        f_m00_B_list.append(current_f_m00_B)
        f_m10_B_list.append(current_f_m10_B)
        f_m01_B_list.append(current_f_m01_B)
        f_m11_B_list.append(current_f_m11_B)

        f_m0_C_list.append(current_f_m0_C)
        f_m1_C_list.append(current_f_m1_C)
        f_m0_D_list.append(current_f_m0_D)
        f_m1_D_list.append(current_f_m1_D)  
        f_m00_R1_list.append(f_m00_R1)
        f_m10_R1_list.append(f_m10_R1)
        f_m01_R1_list.append(f_m01_R1)
        f_m11_R1_list.append(f_m11_R1)
        f_m0_R6_list.append(f_m0_R6)
        f_m1_R6_list.append(f_m1_R6)
        
        f_m1_A_list.append(current_f_m10_A + current_f_m01_A)
        f_m1_B_list.append(current_f_m10_B + current_f_m01_B)
                                
        R1_list.append(current_R1)
        R2_list.append(current_R2)
        R3_list.append(current_R3)
        R3_b_list.append(current_R3_b)
        R4_list.append(current_R4)
        R4_b_list.append(current_R4_b)
        R5_list.append(current_R5)
        R6_list.append(current_R6)
    
    R1_net = np.array(R1_list)
    R2_net = np.array(R2_list)
    R3_net = np.array(R3_list) - np.array(R3_b_list)                   
    R4_net = np.array(R4_list) - np.array(R4_b_list)                   
    R5_net = np.array(R5_list) 
    R6_net = np.array(R6_list)
                                
    times_including_initial = np.insert(times, 0, times[0] - timestep)  

    fig, ax = plt.subplots(1, 4, figsize = (16, 5))                      

    ax[0].plot(times_including_initial, A_total_list, label = 'A')       
    ax[0].plot(times_including_initial, B_total_list, label = 'B')       
    ax[0].plot(times_including_initial, C_total_list, label = 'C')       
    ax[0].plot(times_including_initial, D_total_list, label = 'D')     

    ax[0].legend()                                                       
    ax[0].set_title('Total Concentration Over Time', fontweight = 'bold')
    ax[0].set_xlabel('Time', fontweight = 'bold')                       
    ax[0].set_ylabel('Total Concentration', fontweight = 'bold')       

    ax[1].plot(times_including_initial, R1_net, label = 'R1')           
    ax[1].plot(times_including_initial, R2_net, label = 'R2')           
    ax[1].plot(times_including_initial, R3_net, label = 'R3')           
    ax[1].plot(times_including_initial, R4_net, label = 'R4')          
    ax[1].plot(times_including_initial, R5_net, label = 'R5')           
    ax[1].plot(times_including_initial, R6_net, label = 'R6')
    
    ax[1].legend()                                                      
    ax[1].set_title('Fluxes', fontweight = 'bold')                     
    ax[1].set_xlabel('Time', fontweight = 'bold')                       
    ax[1].set_ylabel('Flux', fontweight = 'bold')                       

    ax[2].plot(times_including_initial, R1_list, label = 'R1')          
    ax[2].plot(times_including_initial, R2_list, label = 'R2')          
    ax[2].plot(times_including_initial, R3_list, label = 'R3')          
    ax[2].plot(times_including_initial, R3_b_list, label = 'R3_b')      
    ax[2].plot(times_including_initial, R4_list, label = 'R4')          
    ax[2].plot(times_including_initial, R4_b_list, label = 'R4_b')      
    ax[2].plot(times_including_initial, R5_list, label = 'R5')          
    ax[2].plot(times_including_initial, R6_list, label = 'R6')

    ax[2].legend()                                                       
    ax[2].set_title('Fluxes', fontweight = 'bold')                      
    ax[2].set_xlabel('Time', fontweight = 'bold')                        
    ax[2].set_ylabel('Flux', fontweight = 'bold')                        

    ax[3].plot(times_including_initial, f_m00_A_list, label = 'A m0')     
    ax[3].plot(times_including_initial, f_m1_A_list, label = 'A m1')     
    ax[3].plot(times_including_initial, f_m11_A_list, label = 'A m2')     
    ax[3].plot(times_including_initial, f_m00_B_list, label = 'B m0')     
    ax[3].plot(times_including_initial, f_m1_B_list, label = 'B m1')     
    ax[3].plot(times_including_initial, f_m11_B_list, label = 'B m2')     
    ax[3].plot(times_including_initial, f_m0_C_list, label = 'C m0')     
    ax[3].plot(times_including_initial, f_m1_C_list, label = 'C m1')     
    ax[3].plot(times_including_initial, f_m0_D_list, label = 'D m0')     
    ax[3].plot(times_including_initial, f_m1_D_list, label = 'D m1')     

    ax[3].legend()                                                       
    ax[3].set_title('Fraction Labeled Over Time', fontweight = 'bold')   
    ax[3].set_xlabel('Time', fontweight = 'bold')                        
    ax[3].set_ylabel('Total Concentration', fontweight = 'bold')         
    
    results_dict = {'R1 flux':R1_list[-1],
                    'R2 flux':R2_list[-1],
                    'R3 forward flux':R3_list[-1],
                    'R3 reverse flux':R3_b_list[-1],
                    'R3 net flux':R3_net[-1],
                    'R4 forward flux':R4_list[-1],
                    'R4 reverse flux':R4_b_list[-1],
                    'R4 net flux':R4_net[-1],                    
                    'R5 forward flux':R5_list[-1],
                    'R5 net flux':R5_net[-1],
                    'R6 flux':R6_net[-1],
                    'A m0':f_m00_A_list[-1],
                    'A m1':f_m1_A_list[-1],
                    'A m2':f_m11_A_list[-1],
                    'B m0':f_m00_B_list[-1],
                    'B m1':f_m1_B_list[-1],
                    'B m2':f_m11_B_list[-1],
                    'C m0':f_m0_C_list[-1],
                    'C m1':f_m1_C_list[-1],
                    'D m0':f_m0_D_list[-1],
                    'D m1':f_m1_D_list[-1]}
    pprint(results_dict)


In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface.
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements(total_time_slider_value = 1000, timestep_slider_value = 0.1,                   
                       m00_A_slider_value = 0, m10_A_slider_value = 0, m01_A_slider_value = 0, m11_A_slider_value = 0,
                       m00_B_slider_value = 0, m10_B_slider_value = 0, m01_B_slider_value = 0, m11_B_slider_value = 0,
                       m0_C_slider_value = 0, m1_C_slider_value = 0,
                       m0_D_slider_value = 0, m1_D_slider_value = 0,
                       R1_slider_value = 10,
                       K_R2_slider_value = 0.1, K_R3_slider_value = 0.1, K_R3_b_slider_value = 0.001,
                       K_R4_slider_value = 0.01, K_R4_b_slider_value = 0.1,
                       K_R5_slider_value = 0.1, R6_slider_value = 1, f_m00_R1_slider_value = 0, f_m10_R1_slider_value = 1, f_m01_R1_slider_value = 0, f_m11_R1_slider_value = 0, f_m0_R6_slider_value = 0, f_m1_R6_slider_value = 1):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 10000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'm00_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m00_A_slider_value),
                              'm10_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m10_A_slider_value),
                              'm01_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m01_A_slider_value),
                              'm11_A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m11_A_slider_value),
                              'm00_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m00_B_slider_value),
                              'm10_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m10_B_slider_value),
                              'm01_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m01_B_slider_value),
                              'm11_B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m11_B_slider_value),
                              'm0_C_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m0_C_slider_value),
                              'm1_C_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m1_C_slider_value),
                              'm0_D_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m0_D_slider_value),
                              'm1_D_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = m1_D_slider_value),
                              'R1_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = R1_slider_value),
                              'K_R2_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R2_slider_value),                              
                              'K_R3_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R3_slider_value),
                              'K_R3_b_slider':widgets.FloatSlider(min = 0, max = 0.01, step = 0.001, value = K_R3_b_slider_value),
                              'K_R4_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R4_slider_value),
                              'K_R4_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R4_b_slider_value),
                              'K_R5_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = K_R5_slider_value),
                              'R6_slider':widgets.IntSlider(min = 0, max = 100, step = 1, value = R6_slider_value),
                              'f_m00_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m00_R1_slider_value),
                              'f_m10_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m10_R1_slider_value),
                              'f_m01_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m01_R1_slider_value),
                              'f_m11_R1_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m11_R1_slider_value),
                              'f_m0_R6_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m0_R6_slider_value),
                              'f_m1_R6_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = f_m1_R6_slider_value),                             
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements


def on_button_clicked(_):                        
    with out:
        clear_output()
        kinetic_simulation_altered_interactive()
        show_inline_matplotlib_plots()
        
def kinetic_simulation_altered_interactive():                             
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    m00_A = elements['m00_A_slider'].value
    m10_A = elements['m10_A_slider'].value
    m01_A = elements['m01_A_slider'].value
    m11_A = elements['m11_A_slider'].value
    m00_B = elements['m00_B_slider'].value
    m10_B = elements['m10_B_slider'].value
    m01_B = elements['m01_B_slider'].value
    m11_B = elements['m11_B_slider'].value
    m0_C = elements['m0_C_slider'].value
    m1_C = elements['m1_C_slider'].value
    m0_D = elements['m0_D_slider'].value
    m1_D = elements['m1_D_slider'].value
    R1 = elements['R1_slider'].value
    K_R2 = elements['K_R2_slider'].value
    K_R3 = elements['K_R3_slider'].value
    K_R3_b = elements['K_R3_b_slider'].value
    K_R4 = elements['K_R4_slider'].value
    K_R4_b = elements['K_R4_b_slider'].value
    K_R5 = elements['K_R5_slider'].value
    R6 = elements['R6_slider'].value
    f_m00_R1 = elements['f_m00_R1_slider'].value
    f_m10_R1 = elements['f_m10_R1_slider'].value
    f_m01_R1 = elements['f_m01_R1_slider'].value
    f_m11_R1 = elements['f_m11_R1_slider'].value
    f_m0_R6 = elements['f_m0_R6_slider'].value
    f_m1_R6 = elements['f_m1_R6_slider'].value


    kinetic_simulation_altered(total_time, timestep, m00_A, m10_A, m01_A, m11_A, m00_B, m10_B, m01_B, m11_B, m0_C, m1_C, m0_D, m1_D,  
        R1, K_R2, K_R3, K_R3_b, K_R4, K_R4_b,
        K_R5, R6, f_m00_R1, f_m10_R1, f_m01_R1, f_m11_R1, f_m0_R6, f_m1_R6)    

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements()                      

elements['button'].on_click(on_button_clicked)        
                                                       
out = widgets.Output()                                

grid = widgets.GridspecLayout(10, 6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Initial [A 00]</b>')
grid[0,5] = elements['m00_A_slider']

grid[1,0] = widgets.HTML('<b>Initial [A 10]</b>')
grid[1,1] = elements['m10_A_slider']
grid[1,2] = widgets.HTML('<b>Initial [A 01]</b>')
grid[1,3] = elements['m01_A_slider']
grid[1,4] = widgets.HTML('<b>Initial [A 11]</b>')
grid[1,5] = elements['m11_A_slider']

grid[2,0] = widgets.HTML('<b>Initial [B 00]</b>')
grid[2,1] = elements['m00_B_slider']
grid[2,2] = widgets.HTML('<b>Initial [B 10]</b>')
grid[2,3] = elements['m10_B_slider']
grid[2,4] = widgets.HTML('<b>Initial [B 01]</b>')
grid[2,5] = elements['m01_B_slider']

grid[3,0] = widgets.HTML('<b>Initial [B 11]</b>')
grid[3,1] = elements['m11_B_slider']
grid[3,2] = widgets.HTML('<b>Initial [C m0]</b>')
grid[3,3] = elements['m0_C_slider']
grid[3,4] = widgets.HTML('<b>Initial [C m1]</b>')
grid[3,5] = elements['m1_C_slider']

grid[4,0] = widgets.HTML('<b>Initial [D m0]</b>')
grid[4,1] = elements['m0_D_slider']
grid[4,2] = widgets.HTML('<b>Initial [D m1]</b>')
grid[4,3] = elements['m1_D_slider']
grid[4,4] = widgets.HTML('<b>R1 Flux</b>')
grid[4,5] = elements['R1_slider']

grid[5,0] = widgets.HTML('<b>R2 Rate Constant</b>')
grid[5,1] = elements['K_R2_slider']
grid[5,2] = widgets.HTML('<b>R3 Rate Constant</b>')
grid[5,3] = elements['K_R3_slider']
grid[5,4] = widgets.HTML('<b>R3_b Rate Constant </b>')
grid[5,5] = elements['K_R3_b_slider']

grid[6,0] = widgets.HTML('<b>R4 Rate Constant</b>')
grid[6,1] = elements['K_R4_slider']
grid[6,2] = widgets.HTML('<b>R4_b Rate Constant </b>')
grid[6,3] = elements['K_R4_b_slider']
grid[6,4] = widgets.HTML('<b>R5 Rate Constant</b>')
grid[6,5] = elements['K_R5_slider']

grid[7,0] = widgets.HTML('<b>R6 Flux</b>')
grid[7,1] = elements['R6_slider']
grid[7,2] = widgets.HTML('<b>R1 00 Fraction</b>')
grid[7,3] = elements['f_m00_R1_slider']
grid[7,4] = widgets.HTML('<b>R1 10 Fraction</b>')
grid[7,5] = elements['f_m10_R1_slider']

grid[8,0] = widgets.HTML('<b>R1 01 Fraction</b>')
grid[8,1] = elements['f_m01_R1_slider']
grid[8,2] = widgets.HTML('<b>R1 11 Fraction</b>')
grid[8,3] = elements['f_m11_R1_slider']
grid[8,4] = widgets.HTML('<b>R6 m0 Fraction</b>')
grid[8,5] = elements['f_m0_R6_slider']

grid[9,0] = widgets.HTML('<b>R6 m1 Fraction</b>')
grid[9,1] = elements['f_m1_R6_slider']
grid[9,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-info">

**Instructions:** Populate the MDV_Template.txt file with the steady-state labeling levels you got from the exercise above. Save the results from the first set of parameters we tested to a file named "MDV_Case4.txt". Then, run the code cell below to perform the MFA analysis.
</div>

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  # This line loads in our model elements.
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                                 # This line sets our model constraints according to them
model.update()                                                                                                                                   # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                                     # This line creates a carbon source object

mdv_observed = model.load_mdv_data('MDV_Case4.txt')                                                                                              # This line loads in our labeling information 
mdv_observed.set_std(0.01, method = 'absolute')                                                                                                  # This line sets the standard deviation of our measurements
#mdv_observed.add_gaussian_noise(0.01)                                                                                                           # This line adds a small amount of random noise to the data
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#01':0.0,'#11':0.0})                                                                          # This line sets our carbon source information
model.set_experiment('ex1',mdv_observed,cs)                                                                                                      # This line defines and sets an "experiment" based on 
                                                                                                                                                 # our carbon source and labeling info
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a bunch of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            # Generating initial states
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    # Adding initial state to results
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  # Each loop we take the best fit we've had so far and use it to start
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case4_FalseModel.csv'

state, flux_opt1 = model.generate_initial_states(1000, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)

record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

<div class="alert alert-block alert-info">

**Instructions:** Now, evaluate that same dataset with the **correct** model.
</div>

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_R6.txt", format = "text")  # This line loads in our model elements.
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                       # This line creates our model from those elements
state_dic = model.load_states("Day04_MFA_Status_R6.csv",format='csv')                                                                   # This line loads the status file elements
model.set_constraints_from_state_dict(state_dic)                                                                                        # This line sets our model constraints according to them
model.update()                                                                                                                          # This line updates the model with the preceding info
cs = model.generate_carbon_source_templete()                                                                                            # This line creates a carbon source object

mdv_observed = model.load_mdv_data('MDV_Case4.txt')                                                                                     # This line loads in our labeling information 
mdv_observed.set_std(0.01, method = 'absolute')                                                                                         # This line sets the standard deviation of our measurements
#mdv_observed.add_gaussian_noise(0.03)                                                                                                  # This line adds a small amount of random noise to the data
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#01':0.0,'#11':0.0})                                                                 # This line sets our carbon source information
cs.set_each_isotopomer('Cin',{'#0':1.0,'#1':0.0})
model.set_experiment('ex1',mdv_observed,cs)                                                                                             # This line defines and sets an "experiment" based on 
                                                                                                                                        # our carbon source and labeling info
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a bunch of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            # Generating initial states
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    # Adding initial state to results
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  # Each loop we take the best fit we've had so far and use it to start
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case4_TrueModel.csv'

state, flux_opt1 = model.generate_initial_states(1000, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)

record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> After getting your results, let's come back together for another whole class discussion.
</div>

# (4.2) Model Selection in the Presence of Noise

Now, we repeat that previous exercise but with slightly noisier data. 

<div class="alert alert-block alert-info">

**Instructions:** Change the marked line of code in both code cells to increase the random noise added to our labeling data from 0.01 to 0.03. Then, run both code cells several times and examine the results, recording the final Residual Sum of Squares for eaching fitting run. 
</div>

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_Alternative.txt", format = "text")  
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                               
state_dic = model.load_states("Day04_MFA_Status_Alt.csv",format='csv')                                                                           
model.set_constraints_from_state_dict(state_dic)                                                                                                 
model.update()                                                                                                                                   
cs = model.generate_carbon_source_templete()                                                                                                     

mdv_observed = model.load_mdv_data('MDV_Case4.txt')                                                                                             
mdv_observed.set_std(0.01, method = 'absolute')                                                                                                 
mdv_observed.add_gaussian_noise(0.03)                                    # Change this line to add more noise! #                                                                                            
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#01':0.0,'#11':0.0})                                                                                    
model.set_experiment('ex1',mdv_observed,cs)                                                                                                     
                                                                                                                                                 
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a bunch of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case4_FalseModel_NoisyData.csv'

state, flux_opt1 = model.generate_initial_states(1000, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)

record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we reload our model and add in labeling information
#-------------------------------------------------------------------------------------------------------------------------------

reactions, reversible, metabolites, target_fragments = mfapy.mfapyio.load_metabolic_model("Day04_Example_MFA_R6.txt", format = "text")  
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible, metabolites, target_fragments)                                                
state_dic = model.load_states("Day04_MFA_Status_R6.csv",format='csv')                                                                           
model.set_constraints_from_state_dict(state_dic)                                                                                                 
model.update()                                                                                                                                   
cs = model.generate_carbon_source_templete()                                                                                                     

mdv_observed = model.load_mdv_data('MDV_Case4.txt')                                                                                             
mdv_observed.set_std(0.01, method = 'absolute')                                                                                                
mdv_observed.add_gaussian_noise(0.03)                             # Change this line to add more noise! #                                                                                                        
cs.set_each_isotopomer('Aex',{'#00':0.0,'#10':1.0,'#01':0.0,'#11':0.0})                                                                                    
cs.set_each_isotopomer('Cin',{'#0':1.0,'#1':0.0})
model.set_experiment('ex1',mdv_observed,cs)                                                                                                      
                                                                                                                                                 
#-------------------------------------------------------------------------------------------------------------------------------
# Here, we generate a bunch of random starting points and pick the one with the lowest sum of squared residuals
#-------------------------------------------------------------------------------------------------------------------------------
                                                                                           
print("Generating 100 random initial states using parallel processing")                                                                      
state, flux_opt1 = model.generate_initial_states(500, 100, method ="parallel")                            
print("RSS of the best initial state is:",model.calc_rss(flux_opt1[0]))                                
results = [('template', flux_opt1[0])]                                                                    
model.set_configuration(callbacklevel = 0) #


#-------------------------------------------------------------------------------------------------------------------------------
# Here, we perform a nonlinear regression procedure to iteratively improve the fit between predicted and measured labeling
#-------------------------------------------------------------------------------------------------------------------------------

print("Fitting by non-linear optimizations")
state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = "SLSQP", flux = flux_opt1)                       
for i in range(10):
    for method in ["SLSQP","LN_SBPLX"]:                                                                  
        state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)
        pvalue, rss_thres = model.goodness_of_fit(flux_opt1[0], alpha = 0.05)
        print("RSS of the best state is:",model.calc_rss(flux_opt1[0]))
    results.append((method, flux_opt1[0]))
model.show_results(results)

#-------------------------------------------------------------------------------------------------------------------------------
# Here we we calculate the 95% confidence invervals on some select reversible reactions
#-------------------------------------------------------------------------------------------------------------------------------

chain_length = 100000
outputfilename = 'MonteCarlo_Case4_TrueModel_NoisyData.csv'

state, flux_opt1 = model.generate_initial_states(1000, 1)
for method in ["SLSQP","LN_SBPLX"]:
    state, RSS_bestfit, flux_opt1 = model.fitting_flux(method = method, flux = flux_opt1)

rss = model.calc_rss(flux_opt1)
if rss < 0.001:
    rss = 0.001
df = mdv_observed.get_number_of_measurement()
sf = stats.chi2.sf(x = rss, df = df)
pdf = stats.chi2.pdf(x = rss, df = df)

print(df, "RSS:",rss, "p-value of chisq test", sf, "pdf", pdf)
if sf < 0.95:
    print("The metabolic model failed to overfit to the isotopoer data")

independent_vector, lbi, ubi, independent_list = model.get_independents(flux_opt1)

record = []
record_temp = [rss]
check, flux_vector, reaction_name = model.check_independents(independent_vector)
record_temp.extend(reaction_name)
record.append(record_temp)


Rm_ind = independent_vector[:]
Rm_ind_record = [Rm_ind] * 1000

progress = 0
unaccepted = 0

while progress < chain_length:
    Rm_ind_next = Rm_ind[:]
    for j in range(100):
        reac_num = numpy.random.randint(0, len(independent_vector))
        Rm_ind_next_temp = Rm_ind_next[:]
        perturbation =  (numpy.random.rand() - 0.5) * 2 * (ubi[reac_num]-lbi[reac_num])/100
        Rm_ind_next_temp[reac_num] += perturbation
        if lbi[reac_num] < Rm_ind_next_temp[reac_num] < ubi[reac_num]:
            check, flux_vector, reaction_name = model.check_independents(Rm_ind_next_temp)
            if check == True:
                Rm_ind_next[reac_num] = Rm_ind_next[reac_num] + perturbation
                break
    else:
        print("Can't find next step. Return to 1000 steps before.")
        Rm_ind = Rm_ind_record[0]
        continue
    rss_next = model.calc_rss(Rm_ind_next, mode = "independent")
    pdf_next = stats.chi2.pdf(x = rss_next, df = df)

    if pdf_next < pdf:
        if numpy.random.rand() > pdf_next/pdf:
                progress = progress + 1
                unaccepted = unaccepted + 1
                Rm_ind_record.pop(0)
                Rm_ind_record.append(Rm_ind)
                continue
    Rm_ind[:] = Rm_ind_next[:]
    rss = rss_next
    pdf = pdf_next
    Rm_ind_record.pop(0)
    Rm_ind_record.append(Rm_ind)
    progress = progress + 1
    if progress % 1000 == 0:
        check, flux_vector, reaction_name = model.check_independents(Rm_ind)
        print(progress, rss, unaccepted, progress,Rm_ind)
        record_temp = [rss]
        record_temp.extend(flux_vector)
        record.append(record_temp)

with open(outputfilename, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') #く
    writer.writerows(record)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> After getting your results, let's come back together for one last whole class discussion.
</div>

# **(5.0)** Incorporating metabolic modeling in your work

How do you envision using metabolic modeling - whether this be kinetic modeling, FBA, or MFA - in your work? 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> If you already have an idea of how you'd like to incorporate metabolic modeling into your work, please share it with the class so we can have a discussion. Depending on the project, one or more of the methods covered over the past four days may be applicable to the project(s) you are interested in. Each method requires different types of information / data and provide different results with associated levels of confidence. 